In [ ]:
from pyspark.sql import functions as F
from pyspark.sql import SparkSession
from scipy import stats
import itertools
import seaborn as sns
import pandas as pd
from matplotlib import pyplot as plt
import plot_timeseries
import os
import pandas as pd
from pyspark.sql import Window
import matplotlib.ticker as ticker
from matplotlib.lines import Line2D
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap
from matplotlib.colors import ListedColormap
from matplotlib.patches import Patch, Rectangle, Circle
from wordcloud import WordCloud
import plot_timeseries

sns.set(font_scale=1.5)
sns.set_style('ticks')
%load_ext autoreload
%autoreload 2

#spark = SparkSession.builder.config("spark.driver.bindAddress","192.168.1.196").getOrCreate()
spark = SparkSession.builder.getOrCreate()

Novelty peaks overlap across resource

In [ ]:
year = 2005
resultsReal,resultsShuffled = [],[]
data = (spark.read.parquet("/Users/mariaf/TargetEngine/results/23.06/noveltyCorrelation/evidenceLink=direct/pairs")       
        .filter((F.col("realYearA")>=year)&(F.col("realYearB")>=year)).toPandas())
for s1, s2 in itertools.product(data.datasourceIdA.unique(), data.datasourceIdA.unique()):
    # real
    x=data[(data.datasourceIdA==s1)&(data.datasourceIdB==s2)][["realYearA","realYearB"]].dropna().realYearA
    y=data[(data.datasourceIdA==s1)&(data.datasourceIdB==s2)][["realYearA","realYearB"]].dropna().realYearB
    try:
        rvalue, pvalue = stats.pearsonr(x, y)
        resultsReal.append([s1, s2,round(rvalue,2), round(pvalue,3), len(x)])
    except ValueError:
        resultsReal.append([s1, s2, None, None, len(x)])
    # shuffled
    x=data[(data.datasourceIdA==s1)&(data.datasourceIdB==s2)][["shuffledYearA","shuffledYearB"]].dropna().shuffledYearA
    y=data[(data.datasourceIdA==s1)&(data.datasourceIdB==s2)][["shuffledYearA","shuffledYearB"]].dropna().shuffledYearB
    try:
        rvalue, pvalue = stats.pearsonr(x, y)
        resultsShuffled.append([s1, s2,round(rvalue,2), round(pvalue,3), len(x)])
    except ValueError:
        resultsShuffled.append([s1, s2, None, None, len(x)])
resultsReal = pd.DataFrame(resultsReal, columns=["datasourceIdA","datasourceIdB","r","p","n"])
resultsShuffled = pd.DataFrame(resultsShuffled, columns=["datasourceIdA","datasourceIdB","r","p","n"])
resultsReal.to_csv("../results/23.06/correlation{}.csv".format(year), index=False)
resultsShuffled.to_csv("../results/23.06/correlationShuffled{}.csv".format(year), index=False)

In [ ]:
# correlation clustermap
year = 2005
data = pd.read_csv("../results/23.06/correlation{}.csv".format(year)).copy()
data = data[(data["p"]<=0.05)]
data = (data
        .pivot(index="datasourceIdA", columns="datasourceIdB", values="r")
        .merge(plot_timeseries.getDatatypeForDatasource(output="dataframe").rename(columns={"datasourceId":"datasourceIdA"}), on="datasourceIdA", how="inner")
        .replace(plot_timeseries.getDatasourceToName(output="dict"))
        .rename(columns=plot_timeseries.getDatasourceToName(output="dict"))
        .set_index("datasourceIdA")
        .replace(plot_timeseries.getDatatypeToName(output="dict"))
        .rename(columns={"datatypeId":"Category"})
)
categories = data.pop("Category")
row_colors = categories.map(plot_timeseries.getDatatypeToColour(by="label"))
plt.figure(figsize=(15,15))
sns.set(font_scale=1.5, style="ticks")
ax = sns.clustermap(data.fillna(0), lw=1, cmap="Blues", square=True, vmin=0, vmax=1, row_colors=row_colors, cbar_pos=(0, .35, .03, .4), cbar_kws={"label":"Pearson correlation coefficient"})
ax.ax_row_dendrogram.set_visible(False)
ax.ax_col_dendrogram.set_visible(False)
ax.ax_heatmap.set_xlabel("")
ax.ax_heatmap.set_ylabel("")
#plt.savefig("../results/23.06/plots/correlations.png", dpi=300, bbox_inches="tight")

In [ ]:
# correlation heatmap annotated
year=2005
data = pd.read_csv("../results/23.06/correlation{}.csv".format(year)).copy()
data["annot"] = data.apply(lambda row: "r={}\np={}\nN={}".format(row.r, row.p, row.n), axis=1)
plt.figure(figsize=(30,30))
ax = sns.heatmap(data.pivot(index="datasourceIdA", columns="datasourceIdB", values="r"), 
            annot=data.pivot(index="datasourceIdA", columns="datasourceIdB", values="annot").fillna(""), fmt="",
            lw=1, cbar=False, cmap="Blues", vmin=0, vmax=1)
ax.set_xlabel("")
ax.set_ylabel("")
plt.savefig("../results/23.06/plots/correlations_annot.png", dpi=300)

In [ ]:
data

In [ ]:
# correlation heatmap for shuffled peaks annotated
data = pd.read_csv("../results/23.06/correlationShuffled{}.csv".format(year)).copy()
data["annot"] = data.apply(lambda row: "r={}\np={}\nN={}".format(row.r, row.p, row.n), axis=1)
plt.figure(figsize=(30,30))
ax = sns.heatmap(data.pivot(index="datasourceIdA", columns="datasourceIdB", values="r"), 
            annot=data.pivot(index="datasourceIdA", columns="datasourceIdB", values="annot").fillna(""), fmt="",
            lw=1, cbar=False, cmap="Blues", vmin=0, vmax=1)
ax.set_xlabel("")
ax.set_ylabel("")
plt.savefig("../results/23.06/plots/correlationsShuffled_annot.png", dpi=300)

Bell plots

In [ ]:
yearsRange = "2000-2005"
data = pd.read_csv("/Users/mariaf/TargetEngine/results/25.03/bell_indirect/years="+yearsRange).replace(plot_timeseries.getDatatypeToName(output="dict"))
data["years"] = data.apply(lambda row: row.years.replace("sum(","").replace("-","−").replace(",",", ")[:-1], axis=1)
data = data[(data.datatypeId!="Pre-clinical phase")&(data.datatypeId!="Known drug")]
datatypes = ["Human genetic data", "Literature", "Animal model", "Affected pathway", "RNA expression","Clinical phase I/II","Clinical phase III","Clinical phase IV"][::-1]

plt.figure()
fig, ax = plt.subplots(figsize=(6,6))
sns.set(font_scale=1.5, style="ticks")
for datatypeId in datatypes:
    sns.lineplot(data=data[data.datatypeId==datatypeId], x="years", y="%withPeak", color="w", lw=6, legend=False, ax=ax, alpha=1)
    sns.lineplot(data=data[(data.datatypeId==datatypeId)], x="years", y="%withPeak", color=plot_timeseries.getDatatypeToColour(by="label")[datatypeId], lw=4, legend=False, ax=ax, alpha=1, label=datatypeId)        

ax.set_ylabel("% of targets with novelty peak")
ax.set_xlabel("Peak year - Approval year")
ax.set_ylim(0,100)
ax.grid(axis="y", lw=.5)
#ax.set_title("{} novel drug targets".format(yearsRange), fontweight="bold")
#ax.set_xticklabels(labels=ax.get_xticklabels(), rotation=40, ha="right")
plt.xticks(rotation=45, ha="right") 
ax.axvline(x=6, color="k", linestyle="--", lw=1)
sns.despine()
plt.savefig("../results/25.03/plots/bell_inidrect_{}.png".format(yearsRange), dpi=300, bbox_inches="tight")

In [ ]:
yearsRange = "2010-2015"
data = pd.read_csv("/Users/mariaf/TargetEngine/results/25.03/bell_indirect/years="+yearsRange).replace(plot_timeseries.getDatatypeToName(output="dict"))
data["years"] = data.apply(lambda row: row.years.replace("sum(","").replace("-","−").replace(",",", ")[:-1], axis=1)
data = data[(data.datatypeId!="Pre-clinical phase")&(data.datatypeId!="Known drug")]
datatypes = ["Human genetic data", "Literature", "Animal model", "Affected pathway", "RNA expression","Clinical phase I/II","Clinical phase III","Clinical phase IV"][::-1]

plt.figure()
fig, ax = plt.subplots(figsize=(6,6))
sns.set(font_scale=1.5, style="ticks")
for datatypeId in datatypes:
    sns.lineplot(data=data[data.datatypeId==datatypeId], x="years", y="%withPeak", color="w", lw=6, legend=False, ax=ax, alpha=1)
    sns.lineplot(data=data[(data.datatypeId==datatypeId)], x="years", y="%withPeak", color=plot_timeseries.getDatatypeToColour(by="label")[datatypeId], lw=4, legend=False, ax=ax, alpha=1, label=datatypeId)        

ax.set_ylabel("% of targets with novelty peak")
ax.set_xlabel("Peak year - Approval year")
ax.set_ylim(0,100)
ax.grid(axis="y", lw=.5)
#ax.set_title("{} novel drug targets".format(yearsRange), fontweight="bold")
#ax.set_xticklabels(labels=ax.get_xticklabels(), rotation=40, ha="right")
plt.xticks(rotation=45, ha="right") 
ax.axvline(x=6, color="k", linestyle="--", lw=1)
sns.despine()
plt.savefig("../results/25.03/plots/bell_inidrect_{}.png".format(yearsRange), dpi=300, bbox_inches="tight")

In [ ]:
yearsRange = "2020-2025"
data = pd.read_csv("/Users/mariaf/TargetEngine/results/25.03/bell_indirect/years="+yearsRange).replace(plot_timeseries.getDatatypeToName(output="dict"))
data["years"] = data.apply(lambda row: row.years.replace("sum(","").replace("-","−").replace(",",", ")[:-1], axis=1)
data = data[(data.datatypeId!="Pre-clinical phase")&(data.datatypeId!="Known drug")]
datatypes = ["Human genetic data", "Literature", "Animal model", "Affected pathway", "RNA expression","Clinical phase I/II","Clinical phase III","Clinical phase IV"][::-1]

plt.figure()
fig, ax = plt.subplots(figsize=(6,6))
sns.set(font_scale=1.5, style="ticks")
for datatypeId in datatypes:
    sns.lineplot(data=data[data.datatypeId==datatypeId], x="years", y="%withPeak", color="w", lw=6, legend=False, ax=ax, alpha=1)
    sns.lineplot(data=data[(data.datatypeId==datatypeId)], x="years", y="%withPeak", color=plot_timeseries.getDatatypeToColour(by="label")[datatypeId], lw=4, legend=False, ax=ax, alpha=1, label=datatypeId)        

ax.set_ylabel("% of targets with novelty peak")
ax.set_xlabel("Peak year - Approval year")
ax.set_ylim(0,100)
ax.grid(axis="y", lw=.5)
#ax.set_title("{} novel drug targets".format(yearsRange), fontweight="bold")
#ax.set_xticklabels(labels=ax.get_xticklabels(), rotation=40, ha="right")
plt.xticks(rotation=45, ha="right") 
ax.axvline(x=6, color="k", linestyle="--", lw=1)
sns.despine()
plt.savefig("../results/25.03/plots/bell_inidrect_{}.png".format(yearsRange), dpi=300, bbox_inches="tight")

In [ ]:
yearsRange = "2000-2005"
data = pd.read_csv("/Users/mariaf/TargetEngine/results/25.03/bell/years="+yearsRange).replace(plot_timeseries.getDatatypeToName(output="dict"))
data["years"] = data.apply(lambda row: row.years.replace("sum(","").replace("-","−").replace(",",", ")[:-1], axis=1)
data = data[(data.datatypeId!="Pre-clinical phase")&(data.datatypeId!="Known drug")]
datatypes = ["Human genetic data", "Literature", "Animal model", "Affected pathway", "RNA expression","Clinical phase I/II","Clinical phase III","Clinical phase IV"][::-1]

plt.figure()
fig, ax = plt.subplots(figsize=(6,6))
sns.set(font_scale=1.5, style="ticks")
for datatypeId in datatypes:
    sns.lineplot(data=data[data.datatypeId==datatypeId], x="years", y="%withPeak", color="w", lw=6, legend=False, ax=ax, alpha=1)
    sns.lineplot(data=data[(data.datatypeId==datatypeId)], x="years", y="%withPeak", color=plot_timeseries.getDatatypeToColour(by="label")[datatypeId], lw=4, legend=False, ax=ax, alpha=1, label=datatypeId)        

ax.set_ylabel("% of targets with novelty peak")
ax.set_xlabel("Peak year - Approval year")
ax.set_ylim(0,100)
ax.grid(axis="y", lw=.5)
#ax.set_title("{} novel drug targets".format(yearsRange), fontweight="bold")
#ax.set_xticklabels(labels=ax.get_xticklabels(), rotation=40, ha="right")
plt.xticks(rotation=45, ha="right") 
ax.axvline(x=6, color="k", linestyle="--", lw=1)
sns.despine()
#plt.savefig("../results/25.03/plots/bell_{}.png".format(yearsRange), dpi=300, bbox_inches="tight")

In [ ]:
yearsRange = "2010-2015"
data = pd.read_csv("/Users/mariaf/TargetEngine/results/25.03/bell/years="+yearsRange).replace(plot_timeseries.getDatatypeToName(output="dict"))
data["years"] = data.apply(lambda row: row.years.replace("sum(","").replace("-","−").replace(",",", ")[:-1], axis=1)
data = data[(data.datatypeId!="Pre-clinical phase")&(data.datatypeId!="Known drug")]
datatypes = ["Human genetic data", "Literature", "Animal model", "Affected pathway", "RNA expression","Clinical phase I/II","Clinical phase III","Clinical phase IV"][::-1]

plt.figure()
fig, ax = plt.subplots(figsize=(6,6))
sns.set(font_scale=1.5, style="ticks")
for datatypeId in datatypes:
    sns.lineplot(data=data[data.datatypeId==datatypeId], x="years", y="%withPeak", color="w", lw=6, legend=False, ax=ax, alpha=1)
    sns.lineplot(data=data[(data.datatypeId==datatypeId)], x="years", y="%withPeak", color=plot_timeseries.getDatatypeToColour(by="label")[datatypeId], lw=4, legend=False, ax=ax, alpha=1, label=datatypeId)        

ax.set_ylabel("% of targets with novelty peak")
ax.set_xlabel("Peak year - Approval year")
ax.set_ylim(0,100)
ax.grid(axis="y", lw=.5)
#ax.set_title("{} novel drug targets".format(yearsRange), fontweight="bold")
#ax.set_xticklabels(labels=ax.get_xticklabels(), rotation=40, ha="right")
plt.xticks(rotation=45, ha="right") 
ax.axvline(x=6, color="k", linestyle="--", lw=1)
sns.despine()
plt.savefig("../results/25.03/plots/bell_{}.png".format(yearsRange), dpi=300, bbox_inches="tight")

In [ ]:
yearsRange = "2020-2025"
data = pd.read_csv("/Users/mariaf/TargetEngine/results/25.03/bell/years="+yearsRange).replace(plot_timeseries.getDatatypeToName(output="dict"))
data["years"] = data.apply(lambda row: row.years.replace("sum(","").replace("-","−").replace(",",", ")[:-1], axis=1)
data = data[(data.datatypeId!="Pre-clinical phase")&(data.datatypeId!="Known drug")]
datatypes = ["Human genetic data", "Literature", "Animal model", "Affected pathway", "RNA expression","Clinical phase I/II","Clinical phase III"][::-1]

plt.figure()
fig, ax = plt.subplots(figsize=(6,6))
sns.set(font_scale=1.5, style="ticks")
for datatypeId in datatypes:
    sns.lineplot(data=data[data.datatypeId==datatypeId], x="years", y="%withPeak", color="w", lw=6, legend=False, ax=ax, alpha=1)
    sns.lineplot(data=data[(data.datatypeId==datatypeId)], x="years", y="%withPeak", color=plot_timeseries.getDatatypeToColour(by="label")[datatypeId], lw=4, legend=False, ax=ax, alpha=1, label=datatypeId)        

ax.set_ylabel("% of targets with novelty peak")
ax.set_xlabel("Peak year - Approval year")
ax.set_ylim(0,100)
ax.grid(axis="y", lw=.5)
#ax.set_title("{} novel drug targets".format(yearsRange), fontweight="bold")
#ax.set_xticklabels(labels=ax.get_xticklabels(), rotation=40, ha="right")
plt.xticks(rotation=45, ha="right") 
ax.axvline(x=6, color="k", linestyle="--", lw=1)
sns.despine()
plt.savefig("../results/25.03/plots/bell_{}.png".format(yearsRange), dpi=300, bbox_inches="tight")

In [ ]:
from matplotlib.ticker import PercentFormatter
for yearsRange in ["2000-2005", "2020-2025"]:
    data = pd.read_csv("/Users/mariaf/TargetEngine/results/25.03/bell_indirect/years="+yearsRange).replace(plot_timeseries.getDatatypeToName(output="dict"))
    data["years"] = data.apply(lambda row: row.years.replace("sum(","").replace("-","−").replace(",",",")[:-1], axis=1)
    data = data[(data.datatypeId!="Pre-clinical phase")&(data.datatypeId!="Known drug")]
    plt.figure(figsize=(7,2.5))
    sns.set(font_scale=1, style="ticks")
    ax = sns.barplot(
        data=data[data.datatypeId.isin(["Literature", "Human genetic data","Clinical phase III","Clinical phase I/II","Other"])],
        x="years", y="%withPeak", hue="datatypeId",
        hue_order=["Human genetic data","Literature","Clinical phase I/II", "Clinical phase III","Other"],
        palette=plot_timeseries.getDatatypeToColour(by="label"), width=1
    )
    ax.grid(axis="y", lw=.5)
    ax.legend(frameon=False, title="", loc="best")
    ax.set_ylabel("Per cent of targets")
    ax.set_xlabel("Peak year - Approval year")
    #ax.set_ylim(0,70)
    ax.axvline(x=5, lw=1, linestyle='--', color='k')
    ax.yaxis.set_major_formatter(PercentFormatter())
    sns.despine()
    ax.set_title("{} novel drug targets".format(yearsRange))
    plt.xticks(rotation=40)
    
    #ax.legend([], frameon=False)

    plt.savefig("../results/25.03/plots/bell_indirect_{}.png".format(yearsRange), bbox_inches="tight", dpi=300)

In [ ]:
from matplotlib.ticker import PercentFormatter
for yearsRange in ["2000-2005", "2020-2025"]:
    data = pd.read_csv("/Users/mariaf/TargetEngine/results/25.03/bell/years="+yearsRange).replace(plot_timeseries.getDatatypeToName(output="dict"))
    data["years"] = data.apply(lambda row: row.years.replace("sum(","").replace("-","−").replace(",",",")[:-1], axis=1)
    data = data[(data.datatypeId!="Pre-clinical phase")&(data.datatypeId!="Known drug")]
    plt.figure(figsize=(7,2.5))
    sns.set(font_scale=1, style="ticks")
    ax = sns.barplot(
        data=data[data.datatypeId.isin(["Literature", "Human genetic data","Clinical phase III","Clinical phase I/II","Other"])],
        x="years", y="%withPeak", hue="datatypeId",
        hue_order=["Human genetic data","Literature","Clinical phase I/II", "Clinical phase III","Other"],
        palette=plot_timeseries.getDatatypeToColour(by="label"), width=1
    )
    ax.grid(axis="y", lw=.5)
    if yearsRange=="2000-2005":
        ax.legend(frameon=True, fancybox=False, edgecolor="k", title="", loc="best")
    else:
        ax.legend([], frameon=False)
    ax.set_ylabel("Per cent of targets")
    ax.set_xlabel("Peak year - Approval year")
    #ax.set_ylim(0,70)
    ax.axvline(x=5, lw=1, linestyle='--', color='k')
    ax.yaxis.set_major_formatter(PercentFormatter())
    sns.despine()
    ax.set_title("{} novel drug targets".format(yearsRange).replace("-","–"))
    plt.xticks(rotation=40)
    
    #ax.legend([], frameon=False)

    plt.savefig("../results/25.03/plots/bell_{}.png".format(yearsRange), bbox_inches="tight", dpi=300)

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.ticker import PercentFormatter
import matplotlib.patches as mpatches


yearsRanges = ["2000-2005", "2020-2025"]

sns.set(font_scale=1.2, style="ticks")
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4), sharex=True, sharey=True)

for ax, yearsRange in zip([ax1, ax2], yearsRanges):
    data = pd.read_csv(f"/Users/mariaf/TargetEngine/results/25.03/bell/years={yearsRange}").replace(plot_timeseries.getDatatypeToName(output="dict"))
    data["years"] = data.apply(lambda row: row.years.replace("sum(","").replace("-","−").replace(",",",")[:-1], axis=1)
    data = data[(data.datatypeId!="Pre-clinical phase") & (data.datatypeId!="Known drug")]
    sns.barplot(
        data=data[data.datatypeId.isin(["Literature", "Human genetic data","Clinical phase III","Clinical phase I/II","Other"])],
        x="years", y="%withPeak", hue="datatypeId",
        hue_order=["Clinical phase I/II", "Clinical phase III","Human genetic data","Literature","Other"],
        palette=plot_timeseries.getDatatypeToColour(by="label"), width=1, ax=ax
    )
    ax.grid(axis="y", lw=.5)
    ax.set_ylabel("Per cent of targets")
    ax.set_xlabel(None)
    ax.set_ylim(0,60)
    ax.axvline(x=5, lw=1, linestyle='--', color='k')
    ax.yaxis.set_major_formatter(PercentFormatter())
    sns.despine(ax=ax)
    ax.set_title(f"{yearsRange.replace('-','–')} novel drug targets")
    plt.setp(ax.get_xticklabels(), rotation=40)
    # Remove legend
    if yearsRange=="2000-2005": 
        ax.legend_.remove()
    else:
        ax.legend(title="", fancybox=0, edgecolor="k", frameon=1, handleheight=1, handlelength=1, borderaxespad=0.2, loc="upper right")
#fig.legend(title="", fancybox=0, edgecolor="k", frameon=1)

# Shared axis labels
fig.supxlabel("Peak year - Approval year", fontsize=15, y=0.05)

plt.tight_layout()
plt.savefig("../results/25.03/plots/bell.png", bbox_inches="tight", dpi=300)
plt.show()


In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.ticker import PercentFormatter
import matplotlib.patches as mpatches


yearsRanges = ["2000-2005", "2020-2025"]

sns.set(font_scale=1.2, style="ticks")
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4), sharex=True, sharey=True)

for ax, yearsRange in zip([ax1, ax2], yearsRanges):
    data = pd.read_csv(f"/Users/mariaf/TargetEngine/results/25.03/bell_indirect/years={yearsRange}").replace(plot_timeseries.getDatatypeToName(output="dict"))
    data["years"] = data.apply(lambda row: row.years.replace("sum(","").replace("-","−").replace(",",",")[:-1], axis=1)
    data = data[(data.datatypeId!="Pre-clinical phase") & (data.datatypeId!="Known drug")]
    sns.barplot(
        data=data[data.datatypeId.isin(["Literature", "Human genetic data","Clinical phase III","Clinical phase I/II","Other"])],
        x="years", y="%withPeak", hue="datatypeId",
        hue_order=["Clinical phase I/II", "Clinical phase III","Human genetic data","Literature","Other"],
        palette=plot_timeseries.getDatatypeToColour(by="label"), width=1, ax=ax
    )
    ax.grid(axis="y", lw=.5)
    ax.set_ylabel("Per cent of targets")
    ax.set_xlabel(None)
    ax.set_ylim(0,60)
    ax.axvline(x=5, lw=1, linestyle='--', color='k')
    ax.yaxis.set_major_formatter(PercentFormatter())
    sns.despine(ax=ax)
    ax.set_title(f"{yearsRange.replace('-','–')} novel drug targets")
    plt.setp(ax.get_xticklabels(), rotation=40)
    # Remove legend
    if yearsRange=="2000-2005": 
        ax.legend_.remove()
    else:
        ax.legend(title="", fancybox=0, edgecolor="k", frameon=1, handleheight=1, handlelength=1, borderaxespad=0.2, loc="upper right")
#fig.legend(title="", fancybox=0, edgecolor="k", frameon=1)

# Shared axis labels
fig.supxlabel("Peak year - Approval year", fontsize=15, y=0.05)

plt.tight_layout()
plt.savefig("../results/25.03/plots/bell_indirect.png", bbox_inches="tight", dpi=300)
plt.show()

In [ ]:
data = pd.read_csv("/Users/mariaf/TargetEngine/results/25.03/bell/years="+"2000-2005").replace(plot_timeseries.getDatatypeToName(output="dict"))
data

In [ ]:
# yearsRange = "2020-2022"
# data = pd.read_csv("/Users/mariaf/TargetEngine/results/25.03/noveltyBenchmark/bells/years="+yearsRange).replace(plot_timeseries.getDatatypeToName(output="dict"))
# data["years"] = data.apply(lambda row: row.years.replace("sum(","").replace("-","−").replace(",",", ")[:-1], axis=1)
# data = data[(data.datatypeId!="Pre-clinical phase")&(data.datatypeId!="Known drug")]
# datatypes = ["Human genetic data", "Literature", "Animal model", "Affected pathway", "RNA expression","Clinical phase I/II","Clinical phase III","Clinical phase IV"][::-1]

In [ ]:
# plt.figure()
# fig, ax = plt.subplots(figsize=(6,6))
# sns.set(font_scale=1.5, style="ticks")
# for datatypeId in datatypes:
#     sns.lineplot(data=data[data.datatypeId==datatypeId], x="years", y="%withPeak", color="w", lw=10, legend=False, ax=ax, alpha=1)
#     sns.lineplot(data=data[(data.datatypeId==datatypeId)], x="years", y="%withPeak", color=plot_timeseries.getDatatypeToColour(by="label")[datatypeId], lw=7, legend=False, ax=ax, alpha=1, label=datatypeId)        

# ax.set_ylabel("% of targets with novelty peak")
# ax.set_xlabel("Peak year - Approval year")
# ax.set_ylim(-3,100)
# ax.grid(axis="y", lw=.5)
# #ax.set_title("{} novel drug targets".format(yearsRange), fontweight="bold")
# ax.set_xticklabels(labels=ax.get_xticklabels(), rotation=40, ha="right")
# ax.axvline(x=6, color="k", linestyle="--", lw=1)
# sns.despine()
# plt.savefig("../results/23.06/plots/bells/{}.png".format(yearsRange), dpi=300, bbox_inches="tight")

Cascade plots

In [ ]:
# data = spark.read.parquet("/Users/mariaf/TargetEngine/results/23.06/noveltyBenchmark/cascadesBoxplot").orderBy("years").select("datatypeId","window","years").toPandas()
# for datatypeId in data.datatypeId.unique():
#     sns.set(font_scale=1.7, style="ticks")
#     fig, ax = plt.subplots(figsize=(5,5))
#     sns.boxplot(data=data[data.datatypeId==datatypeId], x="years", y="window", zorder=1, color=plot_timeseries.getDatatypeToColour()[datatypeId], fliersize=0, boxprops={"edgecolor":"w"}, medianprops={"color":"w"})
    
#     ax.set_xticklabels(ax.get_xticklabels(), rotation=40, ha="right",va="top")
#     ax.set_xlabel("Drug approval year")
#     ax.set_ylabel("Peak year - Approval year")
#     ax.set_title(plot_timeseries.getDatatypeToName(output="dict")[datatypeId]+" peak")
#     ax.set_ylim(25,-25)
#     ax.grid(axis="y", lw=.5, zorder=0)
#     ax.axhline(y=0, lw=1, linestyle="--", color="k", zorder=2)
#     sns.despine()
#     plt.savefig("../results/23.06/plots/cascades/{}.png".format(datatypeId.replace("/","-")), dpi=300, bbox_inches="tight")

In [ ]:
spark.read.parquet("/Users/mariaf/TargetEngine/results/25.03/cascade").select("targetId").distinct().count()

In [ ]:
 data.datatypeId.unique()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import scipy.stats
import matplotlib.ticker as ticker

# Read and prepare your data
data = spark.read.parquet("/Users/mariaf/TargetEngine/results/25.03/cascade") \
    .orderBy("approvalYear") \
    .select("datatypeId","window","approvalYear") \
    .toPandas()

datatype_ids = ["affected_pathway","animal_model","1.0/2.0","3.0","genetic_association","literature","rna_expression","somatic_mutation"]
#datatype_ids = data.datatypeId.unique().sort_values()
ncols = 4
nrows = 2

sns.set(font_scale=1.8, style="ticks")
fig, axes = plt.subplots(
    nrows=nrows, ncols=ncols,
    figsize=(5*ncols, 5*nrows),
    sharex='col', sharey='row'
)

axes = axes.flatten()

for idx, datatypeId in enumerate(datatype_ids):
    ax = axes[idx]
    subset = data[data.datatypeId == datatypeId].groupby(["approvalYear"]).median().window.reset_index()
    p = sns.regplot(
        data=subset,
        x="approvalYear",
        y="window",
        color=plot_timeseries.getDatatypeToColour()["genetic_association"],
        ax=ax
    )
    # Calculate regression statistics
    x = subset["approvalYear"]
    y = subset["window"]
    slope, intercept, r_value, p_value, std_err = scipy.stats.linregress(x, y)
    # Annotate R^2 and p-value
    ax.text(
        0.05, 0.97,
        f'$R^2 = {r_value**2:.2f}$\np = {p_value:.2g}',
        transform=ax.transAxes,
        verticalalignment='top',
        fontsize=14
    )
    ax.set_title(plot_timeseries.getDatatypeToName(output="dict")[datatypeId])
    ax.set_xlabel("")
    ax.set_ylabel("")
    ax.set_ylim(15, -15)
    ax.set_xlim(2000, 2025)
    ax.grid(axis="y", lw=.5, zorder=0)
    ax.axhline(y=0, lw=1, linestyle="--", color="k", zorder=2)
    sns.despine(ax=ax)
    ax.xaxis.set_major_locator(ticker.MultipleLocator(5))

# Hide unused axes
for j in range(idx + 1, nrows * ncols):
    fig.delaxes(axes[j])

fig.supxlabel("Drug approval year", fontsize=20, y=.04)
fig.supylabel("Peak year - Approval year", fontsize=20, x=.02)

plt.tight_layout()
#plt.show()
#plt.savefig("../results/25.03/plots/regression.png",dpi=300, bbox_inches="tight")


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import scipy.stats
import matplotlib.ticker as ticker

# Read and prepare your data
data = spark.read.parquet("/Users/mariaf/TargetEngine/results/25.03/cascade_indirect") \
    .orderBy("approvalYear") \
    .select("datatypeId","window","approvalYear") \
    .toPandas()

datatype_ids = ["affected_pathway","animal_model","1.0/2.0","3.0","genetic_association","literature","rna_expression","somatic_mutation"]
#datatype_ids = data.datatypeId.unique().sort_values()
ncols = 4
nrows = 2

sns.set(font_scale=1.8, style="ticks")
fig, axes = plt.subplots(
    nrows=nrows, ncols=ncols,
    figsize=(5*ncols, 5*nrows),
    sharex='col', sharey='row'
)

axes = axes.flatten()

for idx, datatypeId in enumerate(datatype_ids):
    ax = axes[idx]
    subset = data[data.datatypeId == datatypeId].groupby(["approvalYear"]).median().window.reset_index()
    p = sns.regplot(
        data=subset,
        x="approvalYear",
        y="window",
        color=plot_timeseries.getDatatypeToColour()["genetic_association"],
        ax=ax
    )
    # Calculate regression statistics
    x = subset["approvalYear"]
    y = subset["window"]
    slope, intercept, r_value, p_value, std_err = scipy.stats.linregress(x, y)
    # Annotate R^2 and p-value
    ax.text(
        0.05, 0.97,
        f'$R^2 = {r_value**2:.2f}$\np = {p_value:.2g}',
        transform=ax.transAxes,
        verticalalignment='top',
        fontsize=14
    )
    ax.set_title(plot_timeseries.getDatatypeToName(output="dict")[datatypeId])
    ax.set_xlabel("")
    ax.set_ylabel("")
    ax.set_ylim(15, -15)
    ax.set_xlim(2000, 2025)
    ax.grid(axis="y", lw=.5, zorder=0)
    ax.axhline(y=0, lw=1, linestyle="--", color="k", zorder=2)
    sns.despine(ax=ax)
    ax.xaxis.set_major_locator(ticker.MultipleLocator(5))

# Hide unused axes
for j in range(idx + 1, nrows * ncols):
    fig.delaxes(axes[j])

fig.supxlabel("Drug approval year", fontsize=20, y=.04)
fig.supylabel("Peak year - Approval year", fontsize=20, x=.02)

plt.tight_layout()
#plt.show()
plt.savefig("../results/25.03/plots/regression_indirect.png",dpi=300, bbox_inches="tight")


In [ ]:
plt.figure(figsize=(5,5))
ax=sns.boxplot(data=data[data.datatypeId=="literature"].sort_values("approvalYear"), x="approvalYear", y="window", color="lightblue", fliersize=0, linecolor="k")
ax.axhline(y=0, color="r")

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import scipy.stats
import matplotlib.ticker as ticker

# Read and prepare your data
data = spark.read.parquet("/Users/mariaf/TargetEngine/results/25.03/cascade") \
    .orderBy("approvalYear") \
    .select("datatypeId","window","approvalYear") \
    .toPandas()
data['approvalYear'] = data['approvalYear'].astype(int)

min_year = 2000  # replace with your min year
max_year = 2024  # replace with your max year
all_years = list(range(min_year, max_year + 1))

datatype_ids = ["genetic_association","literature","animal_model","affected_pathway","somatic_mutation","rna_expression","1.0/2.0","3.0"]
#datatype_ids = data.datatypeId.unique().sort_values()
ncols = 4
nrows = 2

sns.set(font_scale=1.8, style="ticks")
fig, axes = plt.subplots(
    nrows=nrows, ncols=ncols,
    figsize=(5*ncols, 5*nrows),
    #sharex=False, 
    sharey="row"
)

axes = axes.flatten()

for idx, datatypeId in enumerate(datatype_ids):
#for idx, datatypeId in enumerate(["affected_pathway","literature","literature","literature","genetic_association","literature","rna_expression","somatic_mutation"]):
    print(datatypeId)
    ax = axes[idx]
    subset = data[data.datatypeId == datatypeId]#.sort_values("approvalYear")

    # Overlay interval line and median dot for each year
    for year in all_years:
        yvals = subset[subset["approvalYear"] == year]["window"].values
        if len(yvals) > 0:
            q1 = np.percentile(yvals, 25)
            q3 = np.percentile(yvals, 75)
            median = np.median(yvals)
            # Draw interval line (IQR)
            ax.plot([year, year], [q1, q3], color=plot_timeseries.getDatasourceToColour()["eva"], linewidth=2, zorder=2)
            # Draw median as dot
            ax.scatter(year, median, color=plot_timeseries.getDatasourceToColour()["eva"], s=50, zorder=3)  # black dot

    ax.set_title(plot_timeseries.getDatatypeToName(output="dict")[datatypeId])
    ax.set_xlabel("")
    ax.set_ylabel("")
    ax.set_ylim(15, -15)
    #ax.set_xlim(2000, 2025)
    ax.grid(axis="y", lw=.5, zorder=0)
    ax.axhline(y=0, lw=2, linestyle="-", color=plot_timeseries.getDatasourceToColour()["chembl"], zorder=2)
    sns.despine(ax=ax)
    ax.xaxis.set_major_locator(ticker.MultipleLocator(5))

# Hide unused axes
for j in range(idx + 1, nrows * ncols):
    fig.delaxes(axes[j])

fig.supxlabel("Drug approval year", fontsize=20, y=.04)
fig.supylabel("Peak year - Approval year", fontsize=20, x=.02)

plt.tight_layout()
#plt.show()
plt.savefig("../results/25.03/plots/boxplot.png",dpi=300, bbox_inches="tight")

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import scipy.stats
import matplotlib.ticker as ticker

# Read and prepare your data
data = spark.read.parquet("/Users/mariaf/TargetEngine/results/25.03/cascade_direct_by_clinical") \
    .orderBy("clinicalYear") \
    .select("datatypeId","window","clinicalYear") \
    .toPandas()
data['clinicalYear'] = data['clinicalYear'].astype(int)

min_year = 2000  # replace with your min year
max_year = 2024  # replace with your max year
all_years = list(range(min_year, max_year + 1))

datatype_ids = ["genetic_association","literature","animal_model","affected_pathway","somatic_mutation","rna_expression"]
#datatype_ids = data.datatypeId.unique().sort_values()
ncols = 3
nrows = 2

sns.set(font_scale=1.8, style="ticks")
fig, axes = plt.subplots(
    nrows=nrows, ncols=ncols,
    figsize=(5*ncols, 5*nrows),
    #sharex=False, 
    sharey="row"
)

axes = axes.flatten()

for idx, datatypeId in enumerate(datatype_ids):
#for idx, datatypeId in enumerate(["affected_pathway","literature","literature","literature","genetic_association","literature","rna_expression","somatic_mutation"]):
    print(datatypeId)
    ax = axes[idx]
    subset = data[data.datatypeId == datatypeId]#.sort_values("approvalYear")

    # Overlay interval line and median dot for each year
    for year in all_years:
        yvals = subset[subset["clinicalYear"] == year]["window"].values
        if len(yvals) > 0:
            q1 = np.percentile(yvals, 25)
            q3 = np.percentile(yvals, 75)
            median = np.median(yvals)
            # Draw interval line (IQR)
            ax.plot([year, year], [q1, q3], color="g", linewidth=2, zorder=2)
            # Draw median as dot
            ax.scatter(year, median, color="g", s=50, zorder=3)  # black dot

    ax.set_title(plot_timeseries.getDatatypeToName(output="dict")[datatypeId])
    ax.set_xlabel("")
    ax.set_ylabel("")
    ax.set_ylim(15, -15)
    #ax.set_xlim(2000, 2025)
    ax.grid(axis="y", lw=.5, zorder=0)
    ax.axhline(y=0, lw=2, linestyle="-", color=plot_timeseries.getDatasourceToColour()["chembl"], zorder=2)
    sns.despine(ax=ax)
    ax.xaxis.set_major_locator(ticker.MultipleLocator(5))

# Hide unused axes
for j in range(idx + 1, nrows * ncols):
    fig.delaxes(axes[j])

fig.supxlabel("Drug approval year", fontsize=20, y=.04)
fig.supylabel("Peak year - Phase I/II clinical trial start year", fontsize=20, x=.02)

plt.tight_layout()
#plt.show()
plt.savefig("../results/25.03/plots/boxplot_clinical.png",dpi=300, bbox_inches="tight")

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import scipy.stats
import matplotlib.ticker as ticker

# Read and prepare your data
data = spark.read.parquet("/Users/mariaf/TargetEngine/results/25.03/cascade") \
    .orderBy("approvalYear") \
    .select("datatypeId","year","approvalYear") \
    .filter(F.col("year").isNotNull()) \
    .toPandas()
data['approvalYear'] = data['approvalYear'].astype(int)
data['year'] = data['year'].astype(int)

min_year = 2000  # replace with your min year
max_year = 2024  # replace with your max year
all_years = list(range(min_year, max_year + 1))

datatype_ids = ["genetic_association","literature","animal_model","affected_pathway","somatic_mutation","rna_expression","1.0/2.0","3.0"]
#datatype_ids = data.datatypeId.unique().sort_values()
ncols = 4
nrows = 2

sns.set(font_scale=1.8, style="ticks")
fig, axes = plt.subplots(
    nrows=nrows, ncols=ncols,
    figsize=(5*ncols, 5*nrows),
    #sharex=False, 
    sharey="row"
)

axes = axes.flatten()

for idx, datatypeId in enumerate(datatype_ids):
#for idx, datatypeId in enumerate(["affected_pathway","literature","literature","literature","genetic_association","literature","rna_expression","somatic_mutation"]):
    print(datatypeId)
    ax = axes[idx]
    subset = data[data.datatypeId == datatypeId]#.sort_values("approvalYear")

    # Overlay interval line and median dot for each year
    for year in all_years:
        yvals = subset[subset["approvalYear"] == year]["year"].values
        if len(yvals) > 0:
            q1 = np.percentile(yvals, 25)
            q3 = np.percentile(yvals, 75)
            median = np.median(yvals)
            # Draw interval line (IQR)
            ax.plot([year, year], [q1, q3], color=plot_timeseries.getDatasourceToColour()["eva"], linewidth=2, zorder=2)
            # Draw median as dot
            ax.scatter(year, median, color=plot_timeseries.getDatasourceToColour()["eva"], s=50, zorder=3)  # black dot

    ax.set_title(plot_timeseries.getDatatypeToName(output="dict")[datatypeId])
    ax.set_xlabel("")
    ax.set_ylabel("")
    ax.set_ylim(15, -15)
    #ax.set_xlim(2000, 2025)
    ax.grid(axis="y", lw=.5, zorder=0)
    ax.axhline(y=0, lw=2, linestyle="-", color=plot_timeseries.getDatasourceToColour()["chembl"], zorder=2)
    sns.despine(ax=ax)
    ax.xaxis.set_major_locator(ticker.MultipleLocator(5))

# Hide unused axes
for j in range(idx + 1, nrows * ncols):
    fig.delaxes(axes[j])

fig.supxlabel("Drug approval year", fontsize=20, y=.04)
fig.supylabel("Peak year - Approval year", fontsize=20, x=.02)

plt.tight_layout()
#plt.show()
#plt.savefig("../results/25.03/plots/boxplot.png",dpi=300, bbox_inches="tight")

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import scipy.stats
import matplotlib.ticker as ticker

# Read and prepare your data
data = spark.read.parquet("/Users/mariaf/TargetEngine/results/25.03/cascade") \
    .orderBy("approvalYear") \
    .select("datatypeId","year","approvalYear") \
    .filter(F.col("year").isNotNull()) \
    .toPandas()
data['approvalYear'] = data['approvalYear'].astype(int)
data['year'] = data['year'].astype(int)

min_year = 2000  # replace with your min year
max_year = 2024  # replace with your max year
all_years = list(range(min_year, max_year + 1))

datatype_ids = ["genetic_association","literature","animal_model","affected_pathway","somatic_mutation","rna_expression","1.0/2.0","3.0"]
#datatype_ids = data.datatypeId.unique().sort_values()
ncols = 4
nrows = 2

sns.set(font_scale=1.8, style="ticks")
fig, axes = plt.subplots(
    nrows=nrows, ncols=ncols,
    figsize=(5*ncols, 5*nrows),
    #sharex=False, 
    sharey="row"
)

axes = axes.flatten()

for idx, datatypeId in enumerate(datatype_ids):
#for idx, datatypeId in enumerate(["affected_pathway","literature","literature","literature","genetic_association","literature","rna_expression","somatic_mutation"]):
    print(datatypeId)
    ax = axes[idx]
    subset = data[data.datatypeId == datatypeId]#.sort_values("approvalYear")

    sns.violinplot(data=subset, x="approvalYear", y="year", color="lightblue", ax=ax)# Overlay interval line and median dot for each year
    
    ax.set_title(plot_timeseries.getDatatypeToName(output="dict")[datatypeId])
    ax.set_xlabel("")
    ax.set_ylabel("")
    #ax.set_ylim(min_year, max_year)
    #ax.set_xlim(min_year, max_year)
    ax.grid(axis="y", lw=.5, zorder=0)
    sns.despine(ax=ax)
    ax.xaxis.set_major_locator(ticker.MultipleLocator(5))

# Hide unused axes
for j in range(idx + 1, nrows * ncols):
    fig.delaxes(axes[j])

fig.supxlabel("Drug approval year", fontsize=20, y=.04)
fig.supylabel("Peak year - Approval year", fontsize=20, x=.02)

plt.tight_layout()
#plt.show()
#plt.savefig("../results/25.03/plots/boxplot.png",dpi=300, bbox_inches="tight")

In [ ]:
data = spark.read.parquet("/Users/mariaf/TargetEngine/results/25.03/cascade_indirect").orderBy("approvalYear").select("datatypeId","window","approvalYear").toPandas()
for datatypeId in data.datatypeId.unique():
    sns.set(font_scale=1.7, style="ticks")
    fig, ax = plt.subplots(figsize=(5,5))
    sns.regplot(data=data[data.datatypeId==datatypeId].groupby(["approvalYear"]).median().window.reset_index(), x="approvalYear", y="window", color=plot_timeseries.getDatatypeToColour()[datatypeId])
    
    #ax.set_xticklabels(ax.get_xticklabels(), rotation=40, ha="right",va="top")
    ax.set_xlabel("Drug approval year")
    ax.set_ylabel("Peak year - Approval year")
    ax.set_title(plot_timeseries.getDatatypeToName(output="dict")[datatypeId])
    ax.set_ylim(15,-15)
    ax.set_xlim(2000,2024)
    ax.grid(axis="y", lw=.5, zorder=0)
    ax.axhline(y=0, lw=1, linestyle="--", color="k", zorder=2)
    sns.despine()
    ax.xaxis.set_major_locator(ticker.MultipleLocator(5))
    #plt.xticks(rotation=45, ha="right")
    plt.savefig("../results/25.03/plots/cascade_indirect_{}.png".format(datatypeId.replace("/","-")), dpi=300, bbox_inches="tight")

In [ ]:
data = spark.read.parquet("/Users/mariaf/TargetEngine/results/25.03/cascadesBoxplot").orderBy("years").select("datatypeId","window","years").toPandas()
for datatypeId in data.datatypeId.unique():
    sns.set(font_scale=1.7, style="ticks")
    fig, ax = plt.subplots(figsize=(5,5))
    sns.boxplot(data=data[data.datatypeId==datatypeId], x="years", y="window", zorder=1, color=plot_timeseries.getDatatypeToColour()[datatypeId], fliersize=0, boxprops={"edgecolor":"w"}, medianprops={"color":"w"})
    
    #ax.set_xticklabels(ax.get_xticklabels(), rotation=40, ha="right",va="top")
    ax.set_xlabel("Drug approval year")
    ax.set_ylabel("Peak year - Approval year")
    ax.set_title(plot_timeseries.getDatatypeToName(output="dict")[datatypeId]+" peak")
    ax.set_ylim(25,-25)
    ax.grid(axis="y", lw=.5, zorder=0)
    ax.axhline(y=0, lw=1, linestyle="--", color="k", zorder=2)
    sns.despine()
    plt.xticks(rotation=45, ha="right")
    #plt.savefig("../results/23.06/plots/cascades/{}.png".format(datatypeId.replace("/","-")), dpi=300, bbox_inches="tight")

Novel drug targets since 2000

In [ ]:
(spark.read.option("header", "true").csv("/Users/mariaf/ChEMBL/CHEMBL_34/drugs.csv")
  .join(spark.read.parquet("/Users/mariaf/OT_platform/23.06/evidence/sourceId=chembl").filter(F.col("clinicalPhase")==4).select("drugId","targetId","diseaseId").distinct(), "drugId", "left")
  .filter(F.col("targetId").isNotNull()&(F.col("firstApprovalYear").isNotNull()))
  .withColumn("minApprovalYear", F.min("firstApprovalYear").over(Window.partitionBy("targetId")))
  .filter(F.col("minApprovalYear")==F.col("firstApprovalYear"))
  .select("drugId","drug","targetId","diseaseId",F.col("firstApprovalYear").alias("approvalYear")).distinct()
).write.parquet("../data/novelDrugTargets/novelDrugTargets")

In [ ]:
(spark.read.option("header", "true").csv("/Users/mariaf/ChEMBL/CHEMBL_35/drugs.csv")
  .join(spark.read.parquet("/Users/mariaf/OT_platform/25.03/evidence/sourceId=chembl").filter(F.col("clinicalPhase")==4).select("drugId","targetId","diseaseId").distinct(), "drugId", "left")
  .filter(F.col("targetId").isNotNull()&(F.col("firstApprovalYear").isNotNull()))
  .withColumn("minApprovalYear", F.min("firstApprovalYear").over(Window.partitionBy("targetId")))
  .filter(F.col("minApprovalYear")==F.col("firstApprovalYear"))
  .select("drugId","drug","targetId","diseaseId",F.col("firstApprovalYear").alias("approvalYear")).distinct()
).write.parquet("/Users/mariaf/TargetEngine/results/25.03/novelDrugTargets/novelDrugTargets")

Timestamps

In [ ]:
# data = (spark.read.parquet("../results/23.06/reportEvidenceDirectDated/")
#         .join(spark.createDataFrame(plot_timeseries.getDatasourceToName(), schema=["datasourceId", "datasourceName"]), "datasourceId", "left")
#         .join(spark.createDataFrame(plot_timeseries.getDatatypeForDatasource(), schema=["datasourceId", "datatypeId"]), "datasourceId", "left")
#         .join(spark.createDataFrame(plot_timeseries.getDatatypeToName(), schema=["datatypeId", "datatypeName"]), "datatypeId", "left")
#         .groupby("datasourceName", "datatypeName").agg(F.sum("nAll").alias("Evidence"))
#         .withColumnRenamed("datasourceName","Source")
# ).toPandas()
# data["sortedDatasourceName"] = data.Source.astype("category")
# data["sortedDatasourceName"] = data.sortedDatasourceName.cat.set_categories(plot_timeseries.datasourceOrder)
# data = data.sort_values("sortedDatasourceName").drop("sortedDatasourceName", axis=1)
# data.sum()

In [ ]:
# plt.figure(figsize=(9, 6))
# sns.set(font_scale=1.5)
# sns.set_style('ticks')
# plt.rcParams['legend.handlelength'] = 1.2
# plt.rcParams['legend.handleheight'] = 1.51

# data = (spark.read.parquet("../results/23.06/reportEvidenceDirectDated/")
#         .filter((F.col("year")<=2023)).select("datasourceId","year")
#         .join(spark.createDataFrame(plot_timeseries.getDatasourceToName(), schema=["datasourceId", "datasourceName"]), "datasourceId", "left")
#         .join(spark.createDataFrame(plot_timeseries.getDatatypeForDatasource(), schema=["datasourceId", "datatypeId"]), "datasourceId", "left")
#         .join(spark.createDataFrame(plot_timeseries.getDatatypeToName(), schema=["datatypeId", "datatypeName"]), "datatypeId", "left")
#         .toPandas())
# data["sortedDatasourceName"] = data.datasourceName.astype("category")
# data["sortedDatasourceName"] = data.sortedDatasourceName.cat.set_categories(plot_timeseries.datasourceOrder)
# data = data.sort_values("sortedDatasourceName").drop("sortedDatasourceName", axis=1)

# ax = sns.boxplot(x="datasourceName", y="year", hue="datatypeName", data=data.replace({"Known drug":"Clinical"}), fliersize=0, width=.6, 
#                  boxprops={"edgecolor":"w"}, medianprops={"color":"k"},
#                  palette=plot_timeseries.getDatatypeToColour(by="label"))
# ax.set_ylim(1950,2024.5)
# ax.set_xlabel("")
# ax.set_ylabel("Year of evidence timestamp")
# ax.set_xticklabels(ax.get_xticklabels(), rotation=50, ha='right')
# ax.spines['top'].set_visible(False)
# ax.spines['right'].set_visible(False)
# ax.grid(axis="y", lw=.5)
# ax.legend(title="Category:", fancybox=False, frameon=True, facecolor="w", edgecolor="k", framealpha=1, fontsize=15, title_fontsize=15)
# plt.savefig("../results/23.06/plots/timestampsBoxplot.png", dpi=300, bbox_inches="tight", transparent=True)

In [ ]:
plt.figure(figsize=(9, 6))
sns.set(font_scale=1.5)
sns.set_style('ticks')
plt.rcParams['legend.handlelength'] = 1.2
plt.rcParams['legend.handleheight'] = 1.51

data = (spark.read.parquet("../results/25.03/evidence_dated/sourceId=gene_burden/")
         .unionByName(spark.read.parquet("../results/25.03/evidence_dated/sourceId=europepmc/").sample(fraction=.01))
         .unionByName(spark.read.parquet("../results/25.03/evidence_dated/sourceId=crispr_screen/"))
         .unionByName(spark.read.parquet("../results/25.03/evidence_dated/sourceId=gene2phenotype/"))
         .unionByName(spark.read.parquet("../results/25.03/evidence_dated/sourceId=chembl/"))
         .unionByName(spark.read.parquet("../results/25.03/evidence_dated/sourceId=slapenrich/"))
         .unionByName(spark.read.parquet("../results/25.03/evidence_dated/sourceId=gwas_credible_sets/"))
         .unionByName(spark.read.parquet("../results/25.03/evidence_dated/sourceId=clingen/"))
         .unionByName(spark.read.parquet("../results/25.03/evidence_dated/sourceId=crispr/"))
         .unionByName(spark.read.parquet("../results/25.03/evidence_dated/sourceId=cancer_biomarkers/"))
         .unionByName(spark.read.parquet("../results/25.03/evidence_dated/sourceId=expression_atlas/"))
         .unionByName(spark.read.parquet("../results/25.03/evidence_dated/sourceId=gene_burden/"))
         .unionByName(spark.read.parquet("../results/25.03/evidence_dated/sourceId=genomics_england/"))
         .unionByName(spark.read.parquet("../results/25.03/evidence_dated/sourceId=orphanet/"))
         .unionByName(spark.read.parquet("../results/25.03/evidence_dated/sourceId=sysbio/"))
         .unionByName(spark.read.parquet("../results/25.03/evidence_dated/sourceId=reactome/"))
         .unionByName(spark.read.parquet("../results/25.03/evidence_dated/sourceId=uniprot_variants/"))
         .unionByName(spark.read.parquet("../results/25.03/evidence_dated/sourceId=impc/").sample(fraction=.01))
         .unionByName(spark.read.parquet("../results/25.03/evidence_dated/sourceId=cancer_gene_census/"))
         .unionByName(spark.read.parquet("../results/25.03/evidence_dated/sourceId=eva/").sample(fraction=.01))
         .unionByName(spark.read.parquet("../results/25.03/evidence_dated/sourceId=eva_somatic/"))
         .unionByName(spark.read.parquet("../results/25.03/evidence_dated/sourceId=uniprot_literature/"))
        
        #.filter((F.col("year")<=2023))
        .select("datasourceId","year","id")
        .join(spark.createDataFrame(plot_timeseries.getDatasourceToName(), schema=["datasourceId", "datasourceName"]), "datasourceId", "left")
        .join(spark.createDataFrame(plot_timeseries.getDatatypeForDatasource(), schema=["datasourceId", "datatypeId"]), "datasourceId", "left")
        .join(spark.createDataFrame(plot_timeseries.getDatatypeToName(), schema=["datatypeId", "datatypeName"]), "datatypeId", "left")
        .toPandas())
data["sortedDatasourceName"] = data.datasourceName.astype("category")
data["sortedDatasourceName"] = data.sortedDatasourceName.cat.set_categories(plot_timeseries.datasourceOrder)
data = data.sort_values("sortedDatasourceName").drop("sortedDatasourceName", axis=1)

ax = sns.boxplot(x="datasourceName", y="year", hue="datatypeName", data=data.replace({"Known drug":"Clinical"}), fliersize=0, width=.6, 
                 boxprops={"edgecolor":"w"}, medianprops={"color":"k"},
                 palette=plot_timeseries.getDatatypeToColour(by="label"))
ax.set_ylim(1970,2028)
ax.set_xlabel("")
ax.set_ylabel("Year of evidence timestamp")
ax.tick_params(axis='x', labelrotation=50)
plt.setp(ax.get_xticklabels(), ha='right')
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.grid(axis="y", lw=.5)
ax.legend(title="", fancybox=False, loc="lower right", ncol=1, frameon=True, facecolor="w", edgecolor="k", framealpha=1, fontsize=14, title_fontsize=15)
#plt.savefig("../results/25.03/plots/timestampsBoxplot.png", dpi=300, bbox_inches="tight", transparent=True)

In [ ]:
# data = (spark.read.parquet("../results/23.06/reportEvidenceDirectDated/")
#         .join(spark.createDataFrame(plot_timeseries.getDatasourceToName(), schema=["datasourceId", "datasourceName"]), "datasourceId", "left")
#         .groupby("datasourceName").agg(F.sum("nDated").alias("Dated evidence"))
#         .withColumnRenamed("datasourceName","Source")
# ).toPandas()
# data["sortedDatasourceName"] = data.Source.astype("category")
# data["sortedDatasourceName"] = data.sortedDatasourceName.cat.set_categories(plot_timeseries.datasourceOrder)
# data = data.sort_values("sortedDatasourceName").drop("sortedDatasourceName", axis=1)
# data

Temporal profiles for case example

In [ ]:
# # evidence
case = (spark.read.parquet("../results/23.06/evidenceIndirectDated/").filter((F.col("diseaseId")=="MONDO_0004979")&(F.col("targetId")=="ENSG00000145777"))
        .join(spark.createDataFrame([["chembl","ChEMBL"],["ot_genetics_portal","OT Genetics"],["europepmc","Europe PMC"]], schema=["datasourceId","datasourceName"]), "datasourceId","left")
 .select("datasourceName","year","score")
).toPandas()

# sns.set(font_scale=1.7, style="ticks")
# fig, ax = plt.subplots(figsize=(5,5))
# sns.scatterplot(data=case, x="year", y="score", hue="datasourceName", palette={"Europe PMC":plot_timeseries.getDatatypeToColour()["literature"],
#                                                                                "OT Genetics":plot_timeseries.getDatatypeToColour()["genetic_association"],
#                                                                                "ChEMBL":plot_timeseries.getDatatypeToColour()["known_drug"]}, hue_order=["Europe PMC","OT Genetics","ChEMBL"], s=500, alpha=.4, ax=ax)
# ax.set_xlim(2000,2024)
# ax.set_ylim(0,1.05)
# ax.set_xlabel("")
# ax.set_ylabel("Score")
# ax.set_title("Evidence")
# ax.xaxis.set_major_locator(ticker.MultipleLocator(base=5))
# ax.legend(frameon=False, loc="upper left")
# sns.despine()
# plt.savefig("../results/23.06/plots/case/evidence.png", dpi=300, bbox_inches="tight")

In [ ]:
spark.read.parquet("/Users/mariaf/TargetEngine/results/25.03/evidence_dated").filter((F.col("diseaseId")=="EFO_0000339")&(F.col("targetId")=="ENSG00000097007")&(F.col("datasourceId")=="europepmc")&(F.col("year")==2009)).withColumn("literature", F.explode("literature")).filter(F.col("literature")==19503090).show()

In [ ]:
spark.read.parquet("/Users/mariaf/TargetEngine/results/25.03/evidence_dated").filter((F.col("diseaseId")=="MONDO_0004979")&(F.col("targetId")=="ENSG00000145777")&(F.col("datasourceId")=="chembl")).show()

In [ ]:
spark.read.parquet("/Users/mariaf/TargetEngine/results/25.03/novelty_by_datasource_dated").filter((F.col("diseaseId")=="MONDO_0004979")&(F.col("targetId")=="ENSG00000145777")&(F.col("datasourceId")=="gwas_credible_sets")&(F.col("year")==2017)).orderBy("year").show()

In [ ]:
spark.read.parquet("../results/23.06/evidenceIndirectDated/").filter((F.col("diseaseId")=="MONDO_0004979")&(F.col("targetId")=="ENSG00000145777")&(F.col("year")==2009)).show()

In [ ]:
spark.read.parquet("../results/23.06/evidenceIndirectDated/").filter((F.col("diseaseId")=="MONDO_0004979")&(F.col("targetId")=="ENSG00000145777")&(F.col("year")==2011)).show()

In [ ]:
spark.read.parquet("../results/23.06/evidenceDated/").select(F.explode("literature").alias("l")).filter(F.col("l")==27019408).show()#.filter(F.col("l")==19557146).show()

In [ ]:
spark.read.parquet("../results/23.06/evidenceDated/")

In [ ]:
spark.read.parquet("../results/23.06/evidenceDated/").filter((F.col("targetId")=="ENSG00000145777")&(F.col("datasourceId")=="europepmc")&(F.col("publicationYear")==2009)&(F.col("score")>=.4)).show()

In [ ]:
spark.read.parquet("../results/23.06/evidenceDated/").select(filter((F.col("targetId")=="ENSG00000145777")&(F.col("datasourceId")=="europepmc")&(F.col("publicationYear")==2009)&(F.col("score")>=.4)).show()

In [ ]:
spark.read.parquet("../results/23.06/evidenceIndirectDated/").filter((F.col("diseaseId")=="MONDO_0004979")&(F.col("targetId")=="ENSG00000145777")&(F.col("datasourceId")=="europepmc")).show()

In [ ]:
from pyspark.sql import functions as F
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

(spark.read.parquet("/Users/mariaf/TargetEngine/results/25.03/evidence_dated")
 .filter((F.col("diseaseId")=="MONDO_0004979")
         &(F.col("targetId")=="ENSG00000145777")
         &(F.col("datasourceId")=="europepmc")
         #&(F.col("year")==2011)
         )
 ).show()

In [ ]:
spark.read.parquet("/Users/mariaf/OT_platform/23.06/evidence/sourceId=europepmc").select(F.explode("literature")).show()#.filter(F.col("col")=='19188585').show()

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.lines as mlines
import matplotlib.patches as mpatches
import matplotlib

sns.set(font_scale=2, style="ticks")
fig, ax = plt.subplots()

# Custom legend elements
bubble1 = mlines.Line2D([], [], color='#AEB236', marker='o', linestyle='None', markersize=20, alpha=.4, markeredgewidth=0, label='Europe PMC evidence')
bubble2 = mlines.Line2D([], [], color='#4280BE', marker='o', linestyle='None', markersize=20, alpha=.4, markeredgewidth=0, label='GWAS evidence')
bubble3 = mlines.Line2D([], [], color='#FF595E', marker='o', linestyle='None', markersize=20, alpha=.4, markeredgewidth=0, label='ChEMBL evidence')

line1 = mlines.Line2D([], [], color='#AEB236', lw=4, label='Europe PMC score')
line2 = mlines.Line2D([], [], color='#4280BE', lw=4, label='GWAS score')
line3 = mlines.Line2D([], [], color='#FF595E', lw=4, label='ChEMBL score')

dashed_line = mlines.Line2D([], [], color='gray', linestyle='--', lw=1, label='Overall score')

# Add legend to the plot
ax.legend(handles=[bubble1, bubble2, bubble3, line1, line2, line3, dashed_line], loc='center left', frameon=True, fancybox=False, edgecolor="k", handlelength=1, handleheight=1)
ax.axis('off')

plt.savefig("../results/25.03/plots/legend1.png", dpi=300, bbox_inches="tight")

In [ ]:
# evidence
case = (spark.read.parquet("../results/25.03/evidence_dated/").filter((F.col("diseaseId")=="MONDO_0004979")&(F.col("targetId")=="ENSG00000145777"))
        .join(spark.createDataFrame([["chembl","ChEMBL"],["gwas_credible_sets","GWAS associations"],["europepmc","Europe PMC"]], schema=["datasourceId","datasourceName"]), "datasourceId","left")
 .select("datasourceName","year","score")
).toPandas()

sns.set(font_scale=2, style="ticks")
fig, ax = plt.subplots(figsize=(5,5))
sns.scatterplot(data=case, x="year", y="score", hue="datasourceName", palette={"Europe PMC":plot_timeseries.getDatatypeToColour()["literature"],
                                                                               "GWAS associations":plot_timeseries.getDatatypeToColour()["genetic_association"],
                                                                               "ChEMBL":plot_timeseries.getDatatypeToColour()["known_drug"]}, hue_order=["GWAS associations","Europe PMC","ChEMBL"], s=500, alpha=.4, ax=ax)
ax.set_xlim(2005,2025)
ax.set_ylim(0,1)
ax.set_xlabel("")
ax.set_ylabel("Score")
ax.set_title("Evidence")
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=5))
ax.legend([], frameon=False, loc="upper left", fontsize=17)
sns.despine()
plt.savefig("../results/25.03/plots/evidence.png", dpi=300, bbox_inches="tight")


In [ ]:
# evidence
case = (spark.read.parquet("../results/25.03/evidence_dated_indirect/").filter((F.col("diseaseId")=="MONDO_0004979")&(F.col("targetId")=="ENSG00000145777"))
        .join(spark.createDataFrame([["chembl","ChEMBL"],["gwas_credible_sets","GWAS associations"],["europepmc","Europe PMC"]], schema=["datasourceId","datasourceName"]), "datasourceId","left")
 .select("datasourceName","year","score")
).toPandas()

sns.set(font_scale=1.7, style="ticks")
fig, ax = plt.subplots(figsize=(5,5))
sns.scatterplot(data=case, x="year", y="score", hue="datasourceName", palette={"Europe PMC":plot_timeseries.getDatatypeToColour()["literature"],
                                                                               "GWAS associations":plot_timeseries.getDatatypeToColour()["genetic_association"],
                                                                               "ChEMBL":plot_timeseries.getDatatypeToColour()["known_drug"]}, hue_order=["GWAS associations","Europe PMC","ChEMBL"], s=500, alpha=.4, ax=ax)
ax.set_xlim(2005,2025)
ax.set_ylim(0,1)
ax.set_xlabel("")
ax.set_ylabel("Score")
ax.set_title("Evidence")
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=5))
ax.legend([], frameon=False, loc="upper left", fontsize=17)
sns.despine()
plt.savefig("../results/25.03/plots/evidence_indirect.png", dpi=300, bbox_inches="tight")

In [ ]:
case[case.year==2011].sort_values(["year"])

In [ ]:
spark.read.parquet("/Users/mariaf/OT_platform/23.06/evidence/sourceId=chembl").filter((F.col("diseaseId")=="MONDO_0004979")&(F.col("targetId")=="ENSG00000145777")&(F.col("clinicalPhase")==4)).select(F.explode("urls"),"studyId","studyStartDate","clinicalPhase","targetId","diseaseId").show(truncate=False)

In [ ]:
spark.read.parquet("/Users/mariaf/OT_platform/25.03/evidence/sourceId=chembl").filter((F.col("diseaseId")=="MONDO_0004979")&(F.col("targetId")=="ENSG00000145777")&(F.col("clinicalPhase")==4)).select(F.explode("urls"),"studyId","studyStartDate","clinicalPhase","targetId","diseaseId").show(truncate=False)

In [ ]:
case[case.year==2008]

In [ ]:
case.groupby("datasourceName").count()

In [ ]:
case.groupby("datasourceName").count()

In [ ]:
spark.read.parquet("/Users/mariaf/TargetEngine/results/25.03/association_by_datasource_dated_indirect").filter((F.col("diseaseId")=="MONDO_0004979")&(F.col("targetId")=="ENSG00000145777")).select("datasourceId").distinct().show()

In [ ]:
# score
sns.set(font_scale=2, style="ticks")
data = (spark.read.parquet("/Users/mariaf/TargetEngine/results/25.03/association_by_datasource_dated").filter((F.col("diseaseId")=="MONDO_0004979")&(F.col("targetId")=="ENSG00000145777"))
.join(spark.createDataFrame([["chembl","ChEMBL"],["gwas_credible_sets","GWAS associations"],["europepmc","Europe PMC"]], schema=["datasourceId","datasourceName"]), "datasourceId","left")
.toPandas().fillna(0))

fig, ax = plt.subplots(figsize=(5,5))
sns.lineplot(data=data, x="year", y="score", hue="datasourceName", palette={"Europe PMC":plot_timeseries.getDatatypeToColour()["literature"],
                                                                               "GWAS associations":plot_timeseries.getDatatypeToColour()["genetic_association"],
                                                                               "ChEMBL":plot_timeseries.getDatatypeToColour()["known_drug"]}, hue_order=["GWAS associations","Europe PMC","ChEMBL"], lw=4)

data = (spark.read.parquet("/Users/mariaf/TargetEngine/results/25.03/association_by_overall_dated").filter((F.col("diseaseId")=="MONDO_0004979")&(F.col("targetId")=="ENSG00000145777"))
.toPandas().fillna(0))
sns.lineplot(data=data, x="year", y="score", color="w", lw=3, linestyle="-", ax=ax)
sns.lineplot(data=data, x="year", y="score", color="grey", lw=1, linestyle="--", label="Overall", ax=ax)


ax.set_xlim(2005,2025)
ax.set_ylim(0,1)
ax.set_xlabel("")
ax.set_ylabel("")
ax.set_yticklabels([])
ax.set_title("Association")
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=5))
plt.rcParams['legend.handlelength'] = 1.5
plt.rcParams['legend.handleheight'] = 1.51
ax.legend([], frameon=False, loc="upper left", fontsize=17)
sns.despine()
plt.savefig("../results/25.03/plots/score.png", dpi=300, bbox_inches="tight")

In [ ]:
# score
sns.set(font_scale=1.7, style="ticks")
data = (spark.read.parquet("/Users/mariaf/TargetEngine/results/25.03/association_by_datasource_dated_indirect").filter((F.col("diseaseId")=="MONDO_0004979")&(F.col("targetId")=="ENSG00000145777"))
.join(spark.createDataFrame([["chembl","ChEMBL"],["gwas_credible_sets","GWAS associations"],["europepmc","Europe PMC"]], schema=["datasourceId","datasourceName"]), "datasourceId","left")
.toPandas().fillna(0))

fig, ax = plt.subplots(figsize=(5,5))
sns.lineplot(data=data, x="year", y="score", hue="datasourceName", palette={"Europe PMC":plot_timeseries.getDatatypeToColour()["literature"],
                                                                               "GWAS associations":plot_timeseries.getDatatypeToColour()["genetic_association"],
                                                                               "ChEMBL":plot_timeseries.getDatatypeToColour()["known_drug"]}, hue_order=["GWAS associations","Europe PMC","ChEMBL"], lw=4)

data = (spark.read.parquet("/Users/mariaf/TargetEngine/results/25.03/association_by_overall_dated_indirect").filter((F.col("diseaseId")=="MONDO_0004979")&(F.col("targetId")=="ENSG00000145777"))
.toPandas().fillna(0))
sns.lineplot(data=data, x="year", y="score", color="w", lw=3, linestyle="-", ax=ax)
sns.lineplot(data=data, x="year", y="score", color="grey", lw=1, linestyle="--", label="Overall", ax=ax)



ax.set_xlim(2005,2025)
ax.set_ylim(0,1)
ax.set_xlabel("")
ax.set_ylabel("Score")
ax.set_title("Association")
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=5))
plt.rcParams['legend.handlelength'] = 1.5
plt.rcParams['legend.handleheight'] = 1.51
ax.legend([], frameon=False, loc="upper left", fontsize=17)
sns.despine()
plt.savefig("../results/25.03/plots/score_indirect.png", dpi=300, bbox_inches="tight")

In [ ]:
data[data.year==0]

In [ ]:
spark.read.parquet("/Users/mariaf/TargetEngine/results/25.03/association_by_overall_dated_indirect").filter((F.col("diseaseId")=="MONDO_0004979")&(F.col("targetId")=="ENSG00000145777")&(F.col("year").isNull())).show()

In [ ]:
spark.read.parquet("/Users/mariaf/TargetEngine/results/25.03/novelty_by_datasource_dated_indirect").filter((F.col("diseaseId")=="MONDO_0004979")&(F.col("targetId")=="ENSG00000145777")&(F.col("year")==2017)).show()

In [ ]:
+0.87/2**2

In [ ]:
(0.8910769493885238/1**2+(0.2882739715330287*0.2)/3**2)/1.64

In [ ]:
data = (spark.read.parquet("/Users/mariaf/TargetEngine/results/25.03/association_by_datasource_dated_indirect").filter((F.col("diseaseId")=="MONDO_0004979")&(F.col("targetId")=="ENSG00000145777"))
.join(spark.createDataFrame([["chembl","ChEMBL"],["gwas_credible_sets","GWAS associations"],["europepmc","Europe PMC"]], schema=["datasourceId","datasourceName"]), "datasourceId","left")
.toPandas().fillna(0))
data[data.year==2017]

In [ ]:
# # score
# sns.set(font_scale=1.7, style="ticks")
# data = (spark.read.parquet("../results/23.06/associationByDatasourceIndirectOverYears/").filter((F.col("diseaseId")=="MONDO_0004979")&(F.col("targetId")=="ENSG00000145777"))
# .join(spark.createDataFrame([["chembl","ChEMBL"],["ot_genetics_portal","OT Genetics"],["europepmc","Europe PMC"]], schema=["datasourceId","datasourceName"]), "datasourceId","left")
# .toPandas())

# fig, ax = plt.subplots(figsize=(5,5))
# sns.lineplot(data=data, x="year", y="score", hue="datasourceName", palette={"Europe PMC":plot_timeseries.getDatatypeToColour()["literature"],
#                                                                                "OT Genetics":plot_timeseries.getDatatypeToColour()["genetic_association"],
#                                                                                "ChEMBL":plot_timeseries.getDatatypeToColour()["known_drug"]}, hue_order=["Europe PMC","OT Genetics","ChEMBL"], lw=3)
# ax.set_xlim(2000,2023)
# ax.set_ylim(0,1.05)
# ax.set_xlabel("")
# ax.set_ylabel("Score")
# ax.set_title("Association")
# ax.xaxis.set_major_locator(ticker.MultipleLocator(base=5))
# plt.rcParams['legend.handlelength'] = 1.5
# plt.rcParams['legend.handleheight'] = 1.51
# ax.legend(frameon=False, loc="upper left")
# sns.despine()
# plt.savefig("../results/23.06/plots/case/score.png", dpi=300, bbox_inches="tight")

In [ ]:
# novelty
sns.set(font_scale=2, style="ticks")
data = (spark.read.parquet("/Users/mariaf/TargetEngine/results/25.03/novelty_by_datasource_dated").filter((F.col("diseaseId")=="MONDO_0004979")&(F.col("targetId")=="ENSG00000145777"))
.join(spark.createDataFrame([["chembl","ChEMBL"],["gwas_credible_sets","GWAS associations"],["europepmc","Europe PMC"]], schema=["datasourceId","datasourceName"]), "datasourceId","left")
.toPandas())

fig, ax = plt.subplots(figsize=(5,5))
sns.lineplot(data=data, x="year", y="novelty", hue="datasourceName", palette={"Europe PMC":plot_timeseries.getDatatypeToColour()["literature"],
                                                                               "GWAS associations":plot_timeseries.getDatatypeToColour()["genetic_association"],
                                                                               "ChEMBL":plot_timeseries.getDatatypeToColour()["known_drug"]},  hue_order=["GWAS associations","Europe PMC","ChEMBL"], lw=4)
data = (spark.read.parquet("/Users/mariaf/TargetEngine/results/25.03/association_by_overall_dated").filter((F.col("diseaseId")=="MONDO_0004979")&(F.col("targetId")=="ENSG00000145777"))
.toPandas().fillna(0))
sns.lineplot(data=data, x="year", y="novelty", color="w", lw=3, linestyle="-", ax=ax)
sns.lineplot(data=data, x="year", y="novelty", color="grey", lw=1, linestyle="--", label="Overall", ax=ax)

ax.set_xlim(2005,2025.1)
ax.set_ylim(0,1)
ax.set_xlabel("")
ax.set_ylabel("")
ax.set_yticklabels([])
ax.set_title("Novelty")
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=5))
plt.rcParams['legend.handlelength'] = 1.5
plt.rcParams['legend.handleheight'] = 1.51
ax.legend(frameon=False, loc="upper left", fontsize=17)
ax.legend([], frameon=False)
sns.despine()
plt.savefig("../results/25.03/plots/novelty.png", dpi=300, bbox_inches="tight")

In [ ]:
# novelty
sns.set(font_scale=1.7, style="ticks")
data = (spark.read.parquet("/Users/mariaf/TargetEngine/results/25.03/novelty_by_datasource_dated_indirect").filter((F.col("diseaseId")=="MONDO_0004979")&(F.col("targetId")=="ENSG00000145777"))
.join(spark.createDataFrame([["chembl","ChEMBL"],["gwas_credible_sets","GWAS associations"],["europepmc","Europe PMC"]], schema=["datasourceId","datasourceName"]), "datasourceId","left")
.toPandas())

fig, ax = plt.subplots(figsize=(5,5))
sns.lineplot(data=data, x="year", y="novelty", hue="datasourceName", palette={"Europe PMC":plot_timeseries.getDatatypeToColour()["literature"],
                                                                               "GWAS associations":plot_timeseries.getDatatypeToColour()["genetic_association"],
                                                                               "ChEMBL":plot_timeseries.getDatatypeToColour()["known_drug"]},  hue_order=["GWAS associations","Europe PMC","ChEMBL"], lw=4)
data = (spark.read.parquet("/Users/mariaf/TargetEngine/results/25.03/association_by_overall_dated_indirect").filter((F.col("diseaseId")=="MONDO_0004979")&(F.col("targetId")=="ENSG00000145777"))
.toPandas().fillna(0))
sns.lineplot(data=data, x="year", y="novelty", color="w", lw=3, linestyle="-", ax=ax)
sns.lineplot(data=data, x="year", y="novelty", color="grey", lw=1, linestyle="--", label="Overall", ax=ax)

ax.set_xlim(2005,2025.1)
ax.set_ylim(0,1)
ax.set_xlabel("")
ax.set_ylabel("Score")
ax.set_title("Novelty")
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=5))
plt.rcParams['legend.handlelength'] = 1.5
plt.rcParams['legend.handleheight'] = 1.51
ax.legend([], frameon=False, loc="upper left", fontsize=17)
sns.despine()
plt.savefig("../results/25.03/plots/novelty_indirect.png", dpi=300, bbox_inches="tight")

In [ ]:
# # novelty
# sns.set(font_scale=1.7, style="ticks")
# data = (spark.read.parquet("../results/23.06/associationByDatasourceIndirectOverYears/").filter((F.col("diseaseId")=="MONDO_0004979")&(F.col("targetId")=="ENSG00000145777"))
# .join(spark.createDataFrame([["chembl","ChEMBL"],["ot_genetics_portal","OT Genetics"],["europepmc","Europe PMC"]], schema=["datasourceId","datasourceName"]), "datasourceId","left")
# .toPandas())

# fig, ax = plt.subplots(figsize=(5,5))
# sns.lineplot(data=data, x="year", y="novelty", hue="datasourceName", palette={"Europe PMC":plot_timeseries.getDatatypeToColour()["literature"],
#                                                                                "OT Genetics":plot_timeseries.getDatatypeToColour()["genetic_association"],
#                                                                                "ChEMBL":plot_timeseries.getDatatypeToColour()["known_drug"]},  hue_order=["Europe PMC","OT Genetics","ChEMBL"], lw=6)
# ax.set_xlim(2000,2023)
# ax.set_ylim(0,1.05)
# ax.set_xlabel("")
# ax.set_ylabel("Score")
# ax.set_title("Novelty")
# ax.xaxis.set_major_locator(ticker.MultipleLocator(base=5))
# plt.rcParams['legend.handlelength'] = 1.5
# plt.rcParams['legend.handleheight'] = 1.51
# ax.legend(frameon=False, loc="upper left")
# sns.despine()
# plt.savefig("../results/23.06/plots/case/novelty.png", dpi=300, bbox_inches="tight")

In [ ]:
spark.read.parquet("/Users/mariaf/OT_platform/25.03/disease").fiselect("therapeuticAreas").show()

In [ ]:
len(ax.get_xticks())

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.lines as mlines
import seaborn as sns
import matplotlib.ticker as ticker

# Set up the figure with 1 row and 4 columns
fig, axes = plt.subplots(1, 4, figsize=(22, 6), gridspec_kw={'width_ratios': [1, 1, 1, 0.8]})
sns.set(font_scale=2.3, style="ticks")

# --- Plot 1: Evidence (Scatter) ---
# # evidence
case = (spark.read.parquet("../results/25.03/evidence_dated/").filter((F.col("diseaseId")=="MONDO_0004979")&(F.col("targetId")=="ENSG00000145777"))
        .join(spark.createDataFrame([["chembl","ChEMBL"],["gwas_credible_sets","GWAS associations"],["europepmc","Europe PMC"]], schema=["datasourceId","datasourceName"]), "datasourceId","left")
 .select("datasourceName","year","score")
).toPandas()

sns.scatterplot(
    data=case, x="year", y="score", hue="datasourceName",
    palette={
        "Europe PMC": plot_timeseries.getDatatypeToColour()["literature"],
        "GWAS associations": plot_timeseries.getDatatypeToColour()["genetic_association"],
        "ChEMBL": plot_timeseries.getDatatypeToColour()["known_drug"]
    },
    hue_order=["GWAS associations", "Europe PMC", "ChEMBL"],
    s=500, alpha=.4, ax=axes[0]
)
axes[0].set_xlim(2005, 2025)
axes[0].set_ylim(0, 1)
axes[0].set_xlabel("")
axes[0].set_ylabel("Score")
axes[0].set_title("Evidence")
axes[0].xaxis.set_major_locator(ticker.MultipleLocator(base=5))
axes[0].legend([], frameon=False)
sns.despine(ax=axes[0])

# --- Plot 2: Association (Line) ---
data_score = (spark.read.parquet("/Users/mariaf/TargetEngine/results/25.03/association_by_datasource_dated").filter((F.col("diseaseId")=="MONDO_0004979")&(F.col("targetId")=="ENSG00000145777"))
.join(spark.createDataFrame([["chembl","ChEMBL"],["gwas_credible_sets","GWAS associations"],["europepmc","Europe PMC"]], schema=["datasourceId","datasourceName"]), "datasourceId","left")
.toPandas().fillna(0))

data_overall = (spark.read.parquet("/Users/mariaf/TargetEngine/results/25.03/association_by_overall_dated").filter((F.col("diseaseId")=="MONDO_0004979")&(F.col("targetId")=="ENSG00000145777"))
.toPandas().fillna(0))

sns.lineplot(
    data=data_score, x="year", y="score", hue="datasourceName",
    palette={
        "Europe PMC": plot_timeseries.getDatatypeToColour()["literature"],
        "GWAS associations": plot_timeseries.getDatatypeToColour()["genetic_association"],
        "ChEMBL": plot_timeseries.getDatatypeToColour()["known_drug"]
    },
    hue_order=["GWAS associations", "Europe PMC", "ChEMBL"],
    lw=4, ax=axes[1]
)
sns.lineplot(data=data_overall, x="year", y="score", color="w", lw=3, linestyle="-", ax=axes[1])
sns.lineplot(data=data_overall, x="year", y="score", color="grey", lw=1, linestyle="--", label="Overall", ax=axes[1])
axes[1].set_xlim(2005, 2025)
axes[1].set_ylim(0, 1)
axes[1].set_xlabel("")
axes[1].set_ylabel("")
axes[1].set_yticklabels([])
axes[1].set_title("Association")
axes[1].xaxis.set_major_locator(ticker.MultipleLocator(base=5))
axes[1].legend([], frameon=False)
sns.despine(ax=axes[1])

# --- Plot 3: Novelty (Line) ---
data_novelty = (spark.read.parquet("/Users/mariaf/TargetEngine/results/25.03/novelty_by_datasource_dated").filter((F.col("diseaseId")=="MONDO_0004979")&(F.col("targetId")=="ENSG00000145777"))
.join(spark.createDataFrame([["chembl","ChEMBL"],["gwas_credible_sets","GWAS associations"],["europepmc","Europe PMC"]], schema=["datasourceId","datasourceName"]), "datasourceId","left")
.toPandas())

data_overall = (spark.read.parquet("/Users/mariaf/TargetEngine/results/25.03/association_by_overall_dated").filter((F.col("diseaseId")=="MONDO_0004979")&(F.col("targetId")=="ENSG00000145777"))
.toPandas().fillna(0))

sns.lineplot(
    data=data_novelty, x="year", y="novelty", hue="datasourceName",
    palette={
        "Europe PMC": plot_timeseries.getDatatypeToColour()["literature"],
        "GWAS associations": plot_timeseries.getDatatypeToColour()["genetic_association"],
        "ChEMBL": plot_timeseries.getDatatypeToColour()["known_drug"]
    },
    hue_order=["GWAS associations", "Europe PMC", "ChEMBL"],
    lw=4, ax=axes[2]
)
sns.lineplot(data=data_overall, x="year", y="novelty", color="w", lw=3, linestyle="-", ax=axes[2])
sns.lineplot(data=data_overall, x="year", y="novelty", color="grey", lw=1, linestyle="--", label="Overall", ax=axes[2])
axes[2].set_xlim(2005, 2025.1)
axes[2].set_ylim(0, 1)
axes[2].set_xlabel("")
axes[2].set_ylabel("")
axes[2].set_yticklabels([])
axes[2].set_title("Novelty")
axes[2].xaxis.set_major_locator(ticker.MultipleLocator(base=5))
axes[2].legend([], frameon=False)
sns.despine(ax=axes[2])

# --- Plot 4: Custom Legend ---
axes[3].axis('off')
bubble1 = mlines.Line2D([], [], color='#AEB236', marker='o', linestyle='None', markersize=20, alpha=.4, markeredgewidth=0, label='Europe PMC evidence')
bubble2 = mlines.Line2D([], [], color='#4280BE', marker='o', linestyle='None', markersize=20, alpha=.4, markeredgewidth=0, label='GWAS evidence')
bubble3 = mlines.Line2D([], [], color='#FF595E', marker='o', linestyle='None', markersize=20, alpha=.4, markeredgewidth=0, label='ChEMBL evidence')
line1 = mlines.Line2D([], [], color='#AEB236', lw=4, label='Europe PMC score')
line2 = mlines.Line2D([], [], color='#4280BE', lw=4, label='GWAS score')
line3 = mlines.Line2D([], [], color='#FF595E', lw=4, label='ChEMBL score')
dashed_line = mlines.Line2D([], [], color='gray', linestyle='--', lw=1, label='Overall score')
axes[3].legend(
    handles=[bubble1, bubble2, bubble3, line1, line2, line3, dashed_line],
    loc='center left', frameon=True, fancybox=False, edgecolor="k", handlelength=1, handleheight=1#, fontsize=17
)

plt.tight_layout()
plt.savefig("../results/25.03/plots/case.png", dpi=300, bbox_inches="tight")
plt.show()


Novelty peaks by data source yearly

In [ ]:
subset

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Suppose your DataFrame is named df
df = (spark.read.parquet("/Users/mariaf/TargetEngine/results/25.03/novelty_distribution/cutoff=0.1").replace("other","z", subset=["datasourceId"]).orderBy("datasourceId","year").replace("z","other", subset=["datasourceId"])
      .filter(F.col("year")<2025)
      #.groupby("datasourceId","year","target").sum("association").withColumnRenamed("sum(association)","association")
      .toPandas()
)
df.target=df.target.replace(0,1)

datasources = df['datasourceId'].replace({"other":"z"}).sort_values().replace({"z":"other"}).unique()
n = len(datasources)
cols = 4  # adjust as needed
rows = (n + cols - 1) // cols

# Set global styles
sns.set(style="ticks", font_scale=1.7)

fig, axes = plt.subplots(rows, cols, figsize=(cols*5, rows*4), squeeze=False, sharex=True, sharey=False)

# Flatten axes for easier indexing
axes_flat = axes.flatten()

# Place legend in the first subplot (axes[0,0])
square_size = 1.0
handles = [
    Rectangle((0,0), square_size, square_size, facecolor=plot_timeseries.getDatatypeToColour()["genetic_association"], label='Non-oncological associations'),
    Rectangle((0,0), square_size, square_size, facecolor=plot_timeseries.getDatatypeToColour()["somatic_mutation"], label='Oncological associations'),
    #Rectangle((0,0), square_size, square_size, facecolor=plot_timeseries.getDatatypeToColour()["somatic_mutation"], label='Other associations'),
    Line2D([0], [0], color=plot_timeseries.getDatatypeToColour()["known_drug"], lw=2, label='Unique targets')
]
axes_flat[0].legend(handles=handles, loc='center', ncol=1, frameon=True, fancybox=False, edgecolor="k", handlelength=1.0,    # Make handle length and height equal
    handleheight=1.0)     # This ensures the handles are square)
axes_flat[0].axis('off')  # Hide the axis for the legend subplot

# Start plotting from the second subplot
for i, datasource in enumerate(datasources):
    #print(datasource)
    ax = axes_flat[i+1]  # Start from the second subplot
    subset = df[df['datasourceId'] == datasource]
    sns.barplot(data=subset.groupby(["year"]).sum().reset_index(), x="year",y="association", width=1, color=plot_timeseries.getDatatypeToColour()["genetic_association"], edgecolor="w", ax=ax)
    sns.barplot(data=subset[subset.therapeuticAreaName=="Oncological"], x="year",y="association",width=1, color=plot_timeseries.getDatatypeToColour()["somatic_mutation"], edgecolor="w", ax=ax)
    #pivot = subset.pivot_table(index='year', columns='therapeuticAreaName', values='association', aggfunc='sum', fill_value=0)
    #pivot = pivot.reindex(columns=["Oncological","Congenital","Other"])
    #pivot.plot(kind='bar', width=1, ax=ax, log=True, legend=False)
    ax.set_title(plot_timeseries.getDatasourceToName(output="dict")[datasource])
    #ax.set_ylim(1, 100000)
    #ax.set_xlim(2000, 2024)
    ax.set_xlim(0, 2025-2000-.5)
    ax.set_xlabel("")
    ax.set_ylabel("")
    #ax.set_xticklabels(ax.get_xticklabels(), rotation=0)
    ax.xaxis.set_major_locator(ticker.MultipleLocator(5))
    #ax.yaxis.set_major_locator(ticker.LogLocator(base=10.0, numticks=10))
    sns.despine(ax=ax)
    # Secondary y-axis line
    #ax.plot(subset.index, subset["target"], color='w', lw=4)
    #ax.plot(subset.groupby(["year"]).max().reset_index().index, subset.groupby(["year"]).max().reset_index()["target"], color='w', lw=5, label='Target')
    ax.plot(subset.groupby(["year"]).max().reset_index().index, subset.groupby(["year"]).max().reset_index()["target"], color=plot_timeseries.getDatatypeToColour()["known_drug"], lw=4, label='Target')

# Hide any unused axes
for j in range(i+2, len(axes_flat)):
    axes_flat[j].axis('off')

# # Add common labels
fig.supxlabel("Year", y=.02)
# fig.supylabel("Log counts with novelty peak")
fig.supylabel("Counts with novelty peak")
sns.despine()

# axes_flat[1].yaxis.set_tick_params(labelleft=True)

plt.tight_layout()
plt.savefig("../results/25.03/plots/novelty_distribution.png", dpi=300, bbox_inches="tight")

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Suppose your DataFrame is named df
df = (spark.read.parquet("/Users/mariaf/TargetEngine/results/25.03/novelty_distribution_indirect/cutoff=0.1").replace("other","z", subset=["datasourceId"]).orderBy("datasourceId","year").replace("z","other", subset=["datasourceId"])
      .filter(F.col("year")<2025)
      #.groupby("datasourceId","year","target").sum("association").withColumnRenamed("sum(association)","association")
      .toPandas()
)
df.target=df.target.replace(0,1)

datasources = df['datasourceId'].replace({"other":"z"}).sort_values().replace({"z":"other"}).unique()
n = len(datasources)
cols = 4  # adjust as needed
rows = (n + cols - 1) // cols

# Set global styles
sns.set(style="ticks", font_scale=1.7)

fig, axes = plt.subplots(rows, cols, figsize=(cols*5, rows*4), squeeze=False, sharex=True, sharey=False)

# Flatten axes for easier indexing
axes_flat = axes.flatten()

# Place legend in the first subplot (axes[0,0])
square_size = 1.0
handles = [
    Rectangle((0,0), square_size, square_size, facecolor=plot_timeseries.getDatatypeToColour()["genetic_association"], label='Non-oncological associations'),
    Rectangle((0,0), square_size, square_size, facecolor=plot_timeseries.getDatatypeToColour()["somatic_mutation"], label='Oncological associations'),
    #Rectangle((0,0), square_size, square_size, facecolor=plot_timeseries.getDatatypeToColour()["somatic_mutation"], label='Other associations'),
    Line2D([0], [0], color=plot_timeseries.getDatatypeToColour()["known_drug"], lw=2, label='Unique targets')
]
axes_flat[0].legend(handles=handles, loc='center', ncol=1, frameon=True, fancybox=False, edgecolor="k", handlelength=1.0,    # Make handle length and height equal
    handleheight=1.0)     # This ensures the handles are square)
axes_flat[0].axis('off')  # Hide the axis for the legend subplot

# Start plotting from the second subplot
for i, datasource in enumerate(datasources):
    #print(datasource)
    ax = axes_flat[i+1]  # Start from the second subplot
    subset = df[df['datasourceId'] == datasource]
    sns.barplot(data=subset.groupby(["year"]).sum().reset_index(), x="year",y="association", width=1, color=plot_timeseries.getDatatypeToColour()["genetic_association"], edgecolor="w", ax=ax)
    sns.barplot(data=subset[subset.therapeuticAreaName=="Oncological"], x="year",y="association",width=1, color=plot_timeseries.getDatatypeToColour()["somatic_mutation"], edgecolor="w", ax=ax)
    #pivot = subset.pivot_table(index='year', columns='therapeuticAreaName', values='association', aggfunc='sum', fill_value=0)
    #pivot = pivot.reindex(columns=["Oncological","Congenital","Other"])
    #pivot.plot(kind='bar', width=1, ax=ax, log=True, legend=False)
    ax.set_title(plot_timeseries.getDatasourceToName(output="dict")[datasource])
    #ax.set_ylim(1, 100000)
    #ax.set_xlim(2000, 2024)
    ax.set_xlim(0, 2025-2000-.5)
    ax.set_xlabel("")
    ax.set_ylabel("")
    #ax.set_xticklabels(ax.get_xticklabels(), rotation=0)
    ax.xaxis.set_major_locator(ticker.MultipleLocator(5))
    #ax.yaxis.set_major_locator(ticker.LogLocator(base=10.0, numticks=10))
    sns.despine(ax=ax)
    # Secondary y-axis line
    #ax.plot(subset.index, subset["target"], color='w', lw=4)
    #ax.plot(subset.groupby(["year"]).max().reset_index().index, subset.groupby(["year"]).max().reset_index()["target"], color='w', lw=5, label='Target')
    ax.plot(subset.groupby(["year"]).max().reset_index().index, subset.groupby(["year"]).max().reset_index()["target"], color=plot_timeseries.getDatatypeToColour()["known_drug"], lw=4, label='Target')

# Hide any unused axes
for j in range(i+2, len(axes_flat)):
    axes_flat[j].axis('off')

# # Add common labels
fig.supxlabel("Year", y=.02)
# fig.supylabel("Log counts with novelty peak")
fig.supylabel("Counts with novelty peak")
sns.despine()

# axes_flat[1].yaxis.set_tick_params(labelleft=True)

plt.tight_layout()
plt.savefig("../results/25.03/plots/novelty_distribution_indirect.png", dpi=300, bbox_inches="tight")

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Suppose your DataFrame is named df
df = (spark.read.parquet("/Users/mariaf/TargetEngine/results/25.03/novelty_distribution_by_datatypeId/cutoff=0.1")
      .filter(F.col("year")<2025)
      .toPandas()
)
df.target=df.target.replace(0,1)

datatypeIds = df['datatypeId'].unique()
n = len(datatypeIds)
cols = 4  # adjust as needed
rows = (n + cols - 1) // cols

# Set global styles
sns.set(style="ticks", font_scale=1.8)

fig, axes = plt.subplots(rows, cols, figsize=(cols*5.6, rows*5), squeeze=False, sharex=False, sharey=False)

# Flatten axes for easier indexing
axes_flat = axes.flatten()

# Place legend in the first subplot (axes[0,0])
square_size = 1.0
handles = [
    Rectangle((0,0), square_size, square_size, facecolor=plot_timeseries.getDatatypeToColour()["genetic_association"], label='Non-oncological associations'),
    Rectangle((0,0), square_size, square_size, facecolor=plot_timeseries.getDatatypeToColour()["somatic_mutation"], label='Oncological associations'),
    #Rectangle((0,0), square_size, square_size, facecolor=plot_timeseries.getDatatypeToColour()["somatic_mutation"], label='Other associations'),
    Line2D([0], [0], color=plot_timeseries.getDatatypeToColour()["known_drug"], lw=2, label='Unique targets')
]
axes_flat[n].legend(handles=handles, loc='center', ncol=1, frameon=True, fancybox=False, edgecolor="k", handlelength=1.0,    # Make handle length and height equal
    handleheight=1.0)     # This ensures the handles are square)
axes_flat[n].axis('off')  # Hide the axis for the legend subplot

datatype_ids=["genetic_association","literature","animal_model","affected_pathway","somatic_mutation","rna_expression","known_drug"]

# Start plotting from the second subplot
for i, datatypeId in enumerate(datatype_ids):
    #print(datasource)
    ax = axes_flat[i]  # Start from the second subplot
    subset = df[df['datatypeId'] == datatypeId]
    sns.barplot(data=subset.groupby(["year"]).sum().reset_index(), x="year",y="association", width=1, color=plot_timeseries.getDatatypeToColour()["genetic_association"], edgecolor="w", ax=ax)
    sns.barplot(data=subset[subset.therapeuticAreaName=="Oncological"], x="year",y="association",width=1, color=plot_timeseries.getDatatypeToColour()["somatic_mutation"], edgecolor="w", ax=ax)
    ax.set_title(plot_timeseries.getDatatypeToName(output="dict")[datatypeId].replace("Known drug","Clinical"))
    if i < 4:
        ax.set_ylim(0, 50000)
    else:
        ax.set_ylim(0, 8000)
    ax.set_xlim(0, 2025-2000-.5)
    ax.set_xlabel("")
    ax.set_ylabel("")
    ax.xaxis.set_major_locator(ticker.MultipleLocator(5))
    sns.despine(ax=ax)
    # Secondary y-axis line
    ax2=ax.twinx()
    #ax.plot(subset.index, subset["target"], color='w', lw=4)
    ax2.plot(subset.groupby(["year"]).max().reset_index().index, subset.groupby(["year"]).max().reset_index()["target"], color='w', lw=4, label='Target')
    ax2.plot(subset.groupby(["year"]).max().reset_index().index, subset.groupby(["year"]).max().reset_index()["target"], color=plot_timeseries.getDatatypeToColour()["known_drug"], lw=3, label='Target')
    if i < 4:
        ax2.set_ylim(0, 4000)
    else:
        ax2.set_ylim(0, 2500)

# Hide any unused axes
for j in range(i+2, len(axes_flat)):
    axes_flat[j].axis('off')

# # Add common labels
fig.supxlabel("Year", y=.05, fontsize=20)
fig.text(.99, 0.5, "Targets with novelty peak", va='center', ha='left', rotation=270, fontsize=20)
# fig.supylabel("Log counts with novelty peak")
fig.supylabel("Associations with novelty peak", fontsize=20)
#sns.despine()

# axes_flat[1].yaxis.set_tick_params(labelleft=True)

plt.tight_layout()
plt.savefig("../results/25.03/plots/novelty_distribution_by_datatype.png", dpi=300, bbox_inches="tight")

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Suppose your DataFrame is named df
df = (spark.read.parquet("/Users/mariaf/TargetEngine/results/25.03/novelty_distribution/cutoff=0.1").replace("other","z", subset=["datasourceId"]).orderBy("datasourceId","year").replace("z","other", subset=["datasourceId"])
      .filter(F.col("year")<2025)
      .groupby("datasourceId","year","target").sum("association").withColumnRenamed("sum(association)","association")
      .toPandas()
)
df.target=df.target.replace(0,1)

datasources = df['datasourceId'].replace({"other":"z"}).sort_values().replace({"z":"other"}).unique()
n = len(datasources)
cols = 4  # adjust as needed
rows = (n + cols - 1) // cols

# Set global styles
sns.set(style="ticks", font_scale=1.7)

fig, axes = plt.subplots(rows, cols, figsize=(cols*5, rows*4), squeeze=False, sharex=True, sharey=True)

# Flatten axes for easier indexing
axes_flat = axes.flatten()

# Place legend in the first subplot (axes[0,0])
square_size = 1.0
handles = [
    Rectangle((0,0), square_size, square_size, facecolor=plot_timeseries.getDatatypeToColour()["genetic_association"], label='Oncological associations'),
    Rectangle((0,0), square_size, square_size, facecolor=plot_timeseries.getDatatypeToColour()["somatic_mutation"], label='Non-oncological associations'),
    #Rectangle((0,0), square_size, square_size, facecolor=plot_timeseries.getDatatypeToColour()["somatic_mutation"], label='Other associations'),
    Line2D([0], [0], color=plot_timeseries.getDatatypeToColour()["known_drug"], lw=2, label='Unique targets')
]
axes_flat[0].legend(handles=handles, loc='center', ncol=1, frameon=True, fancybox=False, edgecolor="k", handlelength=1.0,    # Make handle length and height equal
    handleheight=1.0)     # This ensures the handles are square)
axes_flat[0].axis('off')  # Hide the axis for the legend subplot

# Start plotting from the second subplot
for i, datasource in enumerate(datasources):
    #print(datasource)
    ax = axes_flat[i+1]  # Start from the second subplot
    subset = df[df['datasourceId'] == datasource]
    sns.barplot(data=subset, x="year",y="association",log=True,width=1, color=plot_timeseries.getDatatypeToColour()["genetic_association"], ax=ax)
    sns.barplot(data=subset, x="year",y="association",log=True,width=1, color=plot_timeseries.getDatatypeToColour()["genetic_association"], ax=ax)
    #pivot = subset.pivot_table(index='year', columns='therapeuticAreaName', values='association', aggfunc='sum', fill_value=0)
    #pivot = pivot.reindex(columns=["Oncological","Congenital","Other"])
    #pivot.plot(kind='bar', width=1, ax=ax, log=True, legend=False)
    ax.set_title(plot_timeseries.getDatasourceToName(output="dict")[datasource])
    ax.set_ylim(1, 100000)
    ax.set_xlim(0, 2025-2000-.5)
    ax.set_xlabel("")
    ax.set_ylabel("")
    ax.xaxis.set_major_locator(ticker.MultipleLocator(5))
    ax.yaxis.set_major_locator(ticker.LogLocator(base=10.0, numticks=10))
    sns.despine(ax=ax)
    # Secondary y-axis line
    ax.plot(subset.groupby(["year"]).max().reset_index().index, subset.groupby(["year"]).max().reset_index()["target"], color='w', lw=4, label='Target')
    ax.plot(subset.groupby(["year"]).max().reset_index().index, subset.groupby(["year"]).max().reset_index()["target"], color=plot_timeseries.getDatatypeToColour()["known_drug"], lw=1.5, label='Target')

# Hide any unused axes
for j in range(i+2, len(axes_flat)):
    axes_flat[j].axis('off')

# # Add common labels
fig.supxlabel("Year", y=.02)
fig.supylabel("Log counts with novelty peak")
#fig.supylabel("Counts with novelty peak")
sns.despine()

axes_flat[1].yaxis.set_tick_params(labelleft=True)

plt.tight_layout()
#plt.savefig("../results/25.03/plots/novelty_distribution_log.png", dpi=300, bbox_inches="tight")

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Suppose your DataFrame is named df
df = (spark.read.parquet("/Users/mariaf/TargetEngine/results/25.03/novelty_distribution/cutoff=0.1").replace("other","z", subset=["datasourceId"]).orderBy("datasourceId","year").replace("z","other", subset=["datasourceId"])
      .toPandas()
)
df.target=df.target.replace(0,1)

datasources = df['datasourceId'].replace({"other":"z"}).sort_values().replace({"z":"other"}).unique()
n = len(datasources)
cols = 4  # adjust as needed
rows = (n + cols - 1) // cols

# Set global styles
sns.set(style="ticks", font_scale=1.8)

fig, axes = plt.subplots(rows, cols, figsize=(cols*5, rows*4), squeeze=False, sharex=True, sharey=True)

# Flatten axes for easier indexing
axes_flat = axes.flatten()

# Place legend in the first subplot (axes[0,0])
square_size = 1.0
handles = [
    Rectangle((0,0), square_size, square_size, facecolor=plot_timeseries.getDatatypeToColour()["clinical"], label='Oncological associations'),
    Rectangle((0,0), square_size, square_size, facecolor=plot_timeseries.getDatatypeToColour()["genetic_association"], label='Congenital associations'),
    Rectangle((0,0), square_size, square_size, facecolor=plot_timeseries.getDatatypeToColour()["somatic_mutation"], label='Other associations'),
    Line2D([0], [0], color='k', lw=2, label='Unique targets')
]
axes_flat[0].legend(handles=handles, loc='center', ncol=1, frameon=True, fancybox=False, edgecolor="k", handlelength=1.0,    # Make handle length and height equal
    handleheight=1.0)     # This ensures the handles are square)
axes_flat[0].axis('off')  # Hide the axis for the legend subplot

# Start plotting from the second subplot
for i, datasource in enumerate(datasources):
    ax = axes_flat[i+1]  # Start from the second subplot
    subset = df[df['datasourceId'] == datasource]
    pivot = subset.pivot_table(index='year', columns='therapeuticAreaName', values='association', aggfunc='sum', fill_value=0)
    pivot = pivot.reindex(columns=["Oncological","Congenital","Other"])
    pivot.plot(kind='bar', width=1, stacked=True, ax=ax, log=True, color=[
        plot_timeseries.getDatatypeToColour()["clinical"],
        plot_timeseries.getDatatypeToColour()["genetic_association"],
        plot_timeseries.getDatatypeToColour()["somatic_mutation"]
    ], legend=False)
    ax.set_title(plot_timeseries.getDatasourceToName(output="dict")[datasource])
    ax.set_ylim(1, 100000)
    ax.set_xlabel("")
    ax.set_ylabel("")
    ax.set_xticklabels(ax.get_xticklabels(), rotation=0)
    ax.xaxis.set_major_locator(ticker.MultipleLocator(5))
    ax.yaxis.set_major_locator(ticker.LogLocator(base=10.0, numticks=10))
    #ax.yaxis.set_minor_locator(ticker.LogLocator())
    sns.despine(ax=ax)
    # Secondary y-axis line
    ax.plot(subset.groupby(["year"]).max().reset_index().index, subset.groupby(["year"]).max().reset_index()["target"].replace(0,-1), color='w', lw=4)
    ax.plot(subset.groupby(["year"]).max().reset_index().index, subset.groupby(["year"]).max().reset_index()["target"], color='k', lw=2, label='Target')

# Hide any unused axes
for j in range(i+2, len(axes_flat)):
    axes_flat[j].axis('off')

# Add common labels
fig.supxlabel("Year", y=.02)
fig.supylabel("Log counts with novelty peak")
sns.despine()

axes_flat[1].yaxis.set_tick_params(labelleft=True)

plt.tight_layout()
plt.savefig("../results/25.03/plots/novelty_distribution.png", dpi=300, bbox_inches="tight")

In [ ]:
import pandas as pd

def summarize(group):
    group_sorted = group.sort_values('year').reset_index(drop=True)
    
    total_count = group_sorted['count'].sum()
    idx_max = group_sorted['count'].idxmax()
    year_top = group_sorted.loc[idx_max, 'year']
    count_top = group_sorted.loc[idx_max, 'count']
    
    median_count = group_sorted['count'].median()
    closest_to_median = (group_sorted['count'] - median_count).abs().idxmin()
    year_median = group_sorted.loc[closest_to_median, 'year']
    
    q1 = group_sorted['count'].quantile(0.25)
    q3 = group_sorted['count'].quantile(0.75)
    
    # Year closest to q1 and q3
    closest_to_q1 = (group_sorted['count'] - q1).abs().idxmin()
    year_q1 = group_sorted.loc[closest_to_q1, 'year']
    
    closest_to_q3 = (group_sorted['count'] - q3).abs().idxmin()
    year_q3 = group_sorted.loc[closest_to_q3, 'year']
    
    flatness_kurtosis = group_sorted['count'].kurt()
    
    return pd.Series({
        'total_count': total_count,
        'year_top_count': year_top,
        'top_count': count_top,
        'year_median_count': year_median,
        'q1': q1,
        'year_q1': year_q1,
        'q3': q3,
        'year_q3': year_q3,
        'flatness_kurtosis': flatness_kurtosis
    })

result = data01.groupby('datasourceId').apply(summarize).reset_index()
result.to_csv("../docs/novel_dist_01.tsv",sep="\t", index=False)
result = data05.groupby('datasourceId').apply(summarize).reset_index()
result.to_csv("../docs/novel_dist_05.tsv",sep="\t", index=False)



GE for approvals in Rusina 2023 from OT platform 23.02

In [ ]:
approvalsRusinaGE = pd.read_excel("/Users/mariaf/TargetEngine/data/approvals/26125730.xlsx")#[["drugId","therapeuticArea","yearApproval","hasAnyGE","targetType","diseaseType","brandDrugName"]]


In [ ]:
pd.read_excel("/Users/mariaf/TargetEngine/data/approvals/26125730.xlsx")[["drugId","brandDrugName","therapeuticArea","yearApproval"]].to_csv("../data/approvals.tsv", index=False, sep="\t")
(spark.read.option("header","true").option("sep","\t").csv("../data/approvals.tsv") # Rusina Sup. Table
  .withColumnRenamed("drugId", "drugIds")
  .withColumn("drugId", F.explode_outer(F.split("drugIds", ", ")))
  .join(spark.read.parquet("/Users/mariaf/OT_platform/23.02/evidence/sourceId=chembl").select("diseaseId","targetId","drugId"), ["drugId"], "left")
  .groupby("drugIds","brandDrugName","therapeuticArea","yearApproval").agg(F.collect_set("diseaseId").alias("diseaseIds"), F.collect_set("targetId").alias("targetIds"))
  .withColumnRenamed("drugIds", "drugId")
 ).toPandas().to_csv("approvalsAutomaticPair.tsv", sep="\t", index=False)

Automatic GE extraction from OT_platform 23.02

In [ ]:
approvalsAutomaticGE = (
    # phase IV -> direct target-disease associations since 2013
    spark.read.parquet("/Users/mariaf/OT_platform/23.02/evidence/sourceId=chembl").filter(F.col("clinicalPhase")==4)
    .select("drugId", "diseaseId", "targetId", F.split("studyStartDate", "-").getItem(0).alias("studyStartYear"))
    .filter(F.col("studyStartYear")>=2013)
    # indirect GE
    .join(spark.read.parquet("/Users/mariaf/OT_platform/23.02/evidence/").select("datasourceId","diseaseId","targetId")
          .filter(~F.col("datasourceId").isin(["chembl","europepmc"])), 
          ["diseaseId","targetId"], "left")
)
print("drugId: ", approvalsAutomaticGE.select("drugId").distinct().count())
print("drugId with GE: ", approvalsAutomaticGE.filter(F.col("datasourceId").isNotNull()).select("drugId").distinct().count())
print("% drugId with GE: ", approvalsAutomaticGE.filter(F.col("datasourceId").isNotNull()).select("drugId").distinct().count()/approvalsAutomaticGE.select("drugId").distinct().count())

print("targetId: ", approvalsAutomaticGE.select("targetId").distinct().count())
print("targetId with GE: ", approvalsAutomaticGE.filter(F.col("datasourceId").isNotNull()).select("targetId").distinct().count())
print("% targetId with GE: ", approvalsAutomaticGE.filter(F.col("datasourceId").isNotNull()).select("targetId").distinct().count()/approvalsAutomaticGE.select("targetId").distinct().count())

print("targetId-diseaseId: ", approvalsAutomaticGE.select("targetId","diseaseId").distinct().count())
print("targetId-diseaseId with GE: ", approvalsAutomaticGE.filter(F.col("datasourceId").isNotNull()).select("targetId","diseaseId").distinct().count())
print("% targetId-diseaseId with GE: ", approvalsAutomaticGE.filter(F.col("datasourceId").isNotNull()).select("targetId","diseaseId").distinct().count()/approvalsAutomaticGE.select("targetId","diseaseId").distinct().count())

approvalsAutomaticGE = approvalsAutomaticGE.select("drugId","studyStartYear","datasourceId").distinct().toPandas()
approvalsAutomaticGE["studyStartYear"] = approvalsAutomaticGE.apply(lambda x: int(x.studyStartYear), axis=1)

Rusina section

In [ ]:
approvalsRusinaIndirectGE = pd.read_excel("/Users/mariaf/TargetEngine/data/approvals/26125730.xlsx")[["drugId","therapeuticArea","yearApproval","hasAnyGE","targetType","diseaseType"]]
approvalsRusinaDirectGE = pd.read_excel("/Users/mariaf/TargetEngine/data/approvals/26125730.xlsx")[["drugId","therapeuticArea","yearApproval","hasAnyGE","targetType","diseaseType"]]
approvalsRusinaGE = pd.read_excel("/Users/mariaf/TargetEngine/data/approvals/26125730.xlsx")[["drugId","therapeuticArea","yearApproval","hasAnyGE","targetType","diseaseType"]]
approvalsRusinaGE["therapeuticArea"] = approvalsRusinaGE.apply(lambda row: "oncology" if row.therapeuticArea == "Oncology" else "non-oncology", axis=1)

results = (
    # automatic GE extraction from platform 23.02    
    #approvalsAutomaticGE.groupby("studyStartYear").count().drugId.reset_index().rename(columns={"drugId":"automaticDrugId","studyStartYear":"year"})
    #.merge(approvalsAutomaticGE[approvalsAutomaticGE.datasourceId.notnull()].groupby("studyStartYear").count().drugId.reset_index().rename(columns={"drugId":"automaticGE", "studyStartYear":"year"}), on="year", how="left")
    
    # Rusina approvals
    approvalsRusinaIndirectGE.groupby(["yearApproval"]).count()[["drugId"]].reset_index().rename(columns={"drugId":"Approvals","yearApproval":"year"})
    
    # Rusina approvals supported by indirect GE linking original/interactor target and original/related disease
    .merge(approvalsRusinaGE[(approvalsRusinaGE.hasAnyGE==True)].groupby(["yearApproval"]).count()[["drugId"]].reset_index()
           .rename(columns={"drugId":"Any indirect GE","yearApproval":"year"}), on="year", how="left")      
    # Rusina approvals supported by indirect GE linking original target and original disease
    .merge(approvalsRusinaIndirectGE[(approvalsRusinaIndirectGE.hasAnyGE==True)&(approvalsRusinaIndirectGE.targetType=="Original")&(approvalsRusinaIndirectGE.diseaseType=="Original")].groupby(["yearApproval"]).count()[["drugId"]].reset_index()
           .rename(columns={"drugId":"%Indirect GE between original target and original disease","yearApproval":"year"}), on="year", how="left")
    # Rusina approvals supported by indirect GE linking original target and original/related disease
    .merge(approvalsRusinaIndirectGE[(approvalsRusinaIndirectGE.hasAnyGE==True)&(approvalsRusinaIndirectGE.targetType=="Original")].groupby(["yearApproval"]).count()[["drugId"]].reset_index()
           .rename(columns={"drugId":"Indirect GE between original target and original/related disease","yearApproval":"year"}), on="year", how="left")
    # Rusina approvals supported by indirect GE linking original/interactor target and original disease
    .merge(approvalsRusinaIndirectGE[(approvalsRusinaIndirectGE.hasAnyGE==True)&(approvalsRusinaIndirectGE.diseaseType=="Original")].groupby(["yearApproval"]).count()[["drugId"]].reset_index()
           .rename(columns={"drugId":"Indirect GE between original/interacting target and original disease","yearApproval":"year"}), on="year", how="left")    
    
    # Rusina approvals supported by direct GE linking original/interactor target and original/related disease
    .merge(approvalsRusinaGE[(approvalsRusinaDirectGE.hasAnyGE==True)].groupby(["yearApproval"]).count()[["drugId"]].reset_index()
           .rename(columns={"drugId":"%Direct GE between original/interacting target and original/related disease","yearApproval":"year"}), on="year", how="left")      
    # Rusina approvals supported by direct GE linking original target and original disease
    .merge(approvalsRusinaIndirectGE[(approvalsRusinaDirectGE.hasAnyGE==True)&(approvalsRusinaDirectGE.targetType=="Original")&(approvalsRusinaDirectGE.diseaseType=="Original")].groupby(["yearApproval"]).count()[["drugId"]].reset_index()
           .rename(columns={"drugId":"%Direct GE between original target and original disease","yearApproval":"year"}), on="year", how="left")
    # Rusina approvals supported by direct GE linking original target and original/related disease
    .merge(approvalsRusinaIndirectGE[(approvalsRusinaDirectGE.hasAnyGE==True)&(approvalsRusinaDirectGE.targetType=="Original")].groupby(["yearApproval"]).count()[["drugId"]].reset_index()
           .rename(columns={"drugId":"Direct GE between original target and original/related disease","yearApproval":"year"}), on="year", how="left")
    # Rusina approvals supported by direct GE linking original/interactor target and original disease
    .merge(approvalsRusinaIndirectGE[(approvalsRusinaDirectGE.hasAnyGE==True)&(approvalsRusinaDirectGE.diseaseType=="Original")].groupby(["yearApproval"]).count()[["drugId"]].reset_index()
           .rename(columns={"drugId":"Direct GE between original/interacting target and original disease","yearApproval":"year"}), on="year", how="left")    
)
#results["%automaticGE"] = results["automaticGE"]/results["automaticDrugId"]

results["%Any indirect GE"] = results["Indirect GE between original/interacting target and original/related disease"]/results["Approvals"]
# results["%Indirect GE between original target and original disease"] = results["Indirect GE between original target and original disease"]/results["Approvals"]
# results["%Indirect GE between original target and original/related disease"] = results["Indirect GE between original target and original/related disease"]/results["Approvals"]
# results["%Indirect GE between original/interacting target and original disease"] = results["Indirect GE between original/interacting target and original disease"]/results["Approvals"]

# results["%Direct GE between original/interacting target and original/related disease"] = results["Direct GE between original/interacting target and original/related disease"]/results["Approvals"]
# results["%Direct GE between original target and original disease"] = results["Direct GE between original target and original disease"]/results["Approvals"]
# results["%Direct GE between original target and original/related disease"] = results["Direct GE between original target and original/related disease"]/results["Approvals"]
# results["%Direct GE between original/interacting target and original disease"] = results["Direct GE between original/interacting target and original disease"]/results["Approvals"]


# ax = plt.figure().add_subplot()

# sns.lineplot(data=results, x="year", y="%Indirect GE between original/interacting target and original/related disease", marker=".", markersize=10, color="g", 
#              label="%Indirect GE between original/interacting target and original/related disease", ax=ax)
# sns.lineplot(data=results, x="year", y="Indirect GE between original/interacting target and original disease", marker=".", markersize=10, color="orange", ax=ax, 
#              label="Indirect GE between original/interacting target and original disease")
# sns.lineplot(data=results, x="year", y="Indirect GE between original target and original/related disease", marker=".", markersize=10, color="purple", ax=ax, 
#              label="%Indirect GE between original/interacting target and original disease")
# sns.lineplot(data=results, x="year", y="%Indirect GE between original target and original disease", marker=".", markersize=10, color="b", ax=ax, 
#              label="%Indirect GE between original target and original disease")

# sns.lineplot(data=results, x="year", y="%Direct GE between original/interacting target and original/related disease", marker=".", markersize=10, color="g", 
#              label="%Direct GE between original/interacting target and original/related disease", ax=ax, linestyle="--")
# sns.lineplot(data=results, x="year", y="Direct GE between original/interacting target and original disease", marker=".", markersize=10, color="orange", ax=ax, linestyle="--", 
#              label="Direct GE between original/interacting target and original disease")
# sns.lineplot(data=results, x="year", y="Direct GE between original target and original/related disease", marker=".", markersize=10, color="purple", ax=ax, linestyle="--", 
#              label="%Direct GE between original/interacting target and original disease")
# sns.lineplot(data=results, x="year", y="%Direct GE between original target and original disease", marker=".", markersize=10, color="b", ax=ax, linestyle="--", 
#              label="%Direct GE between original target and original disease")

# #sns.lineplot(data=results, x="year", y="%automaticGE", marker=".", markersize=10, color="r", label="Automatic GE", ax=ax)
# ax.set_ylim(0,1)
# ax.set_xlim(2013,2023)
# ax.set_ylabel("% Approvals")
# ax.set_xlabel("Approval year")
# leg = ax.legend(loc='center left', bbox_to_anchor=(1, 0.5), frameon=0)
# #plt.savefig("../results/23.02/plots/approvals.png", dpi=300, bbox_inches="tight")

In [ ]:
approvalsRusinaIndirectGE = pd.read_excel("/Users/mariaf/TargetEngine/data/approvals/26125730.xlsx")[["drugId","therapeuticArea","yearApproval","hasAnyGE","targetType","diseaseType"]]
approvalsRusinaDirectGE = pd.read_excel("/Users/mariaf/TargetEngine/data/approvals/26125730.xlsx")[["drugId","therapeuticArea","yearApproval","hasAnyGE","targetType","diseaseType"]]
#approvalsRusinaGE["therapeuticArea"] = approvalsRusinaGE.apply(lambda row: "oncology" if row.therapeuticArea == "Oncology" else "non-oncology", axis=1)

results = (
    
    # Rusina approvals
    approvalsRusinaIndirectGE.groupby(["yearApproval"]).count()[["drugId"]].reset_index().rename(columns={"drugId":"Approvals","yearApproval":"year"})
    
    # Rusina approvals supported by direct GE linking original target and original disease
    .merge(approvalsRusinaIndirectGE[(approvalsRusinaDirectGE.hasAnyGE==True)&(approvalsRusinaDirectGE.targetType=="Original")&(approvalsRusinaDirectGE.diseaseType=="Original")].groupby(["yearApproval"]).count()[["drugId"]].reset_index()
           .rename(columns={"drugId":"Direct GE between original target and original disease","yearApproval":"year"}), on="year", how="left")
    
    # Rusina approvals supported by indirect GE linking original target and original disease
    .merge(approvalsRusinaIndirectGE[(approvalsRusinaIndirectGE.hasAnyGE==True)&(approvalsRusinaIndirectGE.targetType=="Original")&(approvalsRusinaIndirectGE.diseaseType=="Original")].groupby(["yearApproval"]).count()[["drugId"]].reset_index()
           .rename(columns={"drugId":"Indirect GE between original target and original disease","yearApproval":"year"}), on="year", how="left")
    
#     # Rusina approvals supported by indirect GE linking original/interactor target and original/related disease
#     .merge(approvalsRusinaGE[(approvalsRusinaIndirectGE.hasAnyGE==True)].groupby(["yearApproval"]).count()[["drugId"]].reset_index()
#            .rename(columns={"drugId":"Indirect GE between original/interacting target and original/related disease","yearApproval":"year"}), on="year", how="left")      
#     # Rusina approvals supported by indirect GE linking original target and original/related disease
#     .merge(approvalsRusinaIndirectGE[(approvalsRusinaIndirectGE.hasAnyGE==True)&(approvalsRusinaIndirectGE.targetType=="Original")].groupby(["yearApproval"]).count()[["drugId"]].reset_index()
#            .rename(columns={"drugId":"Indirect GE between original target and original/related disease","yearApproval":"year"}), on="year", how="left")
#     # Rusina approvals supported by indirect GE linking original/interactor target and original disease
#     .merge(approvalsRusinaIndirectGE[(approvalsRusinaIndirectGE.hasAnyGE==True)&(approvalsRusinaIndirectGE.diseaseType=="Original")].groupby(["yearApproval"]).count()[["drugId"]].reset_index()
#            .rename(columns={"drugId":"Indirect GE between original/interacting target and original disease","yearApproval":"year"}), on="year", how="left")    
    
#     # Rusina approvals supported by direct GE linking original/interactor target and original/related disease
#     .merge(approvalsRusinaGE[(approvalsRusinaDirectGE.hasAnyGE==True)].groupby(["yearApproval"]).count()[["drugId"]].reset_index()
#            .rename(columns={"drugId":"Direct GE between original/interacting target and original/related disease","yearApproval":"year"}), on="year", how="left")      
    
#     # Rusina approvals supported by direct GE linking original target and original/related disease
#     .merge(approvalsRusinaIndirectGE[(approvalsRusinaDirectGE.hasAnyGE==True)&(approvalsRusinaDirectGE.targetType=="Original")].groupby(["yearApproval"]).count()[["drugId"]].reset_index()
#            .rename(columns={"drugId":"Direct GE between original target and original/related disease","yearApproval":"year"}), on="year", how="left")
#     # Rusina approvals supported by direct GE linking original/interactor target and original disease
#     .merge(approvalsRusinaIndirectGE[(approvalsRusinaDirectGE.hasAnyGE==True)&(approvalsRusinaDirectGE.diseaseType=="Original")].groupby(["yearApproval"]).count()[["drugId"]].reset_index()
#            .rename(columns={"drugId":"Direct GE between original/interacting target and original disease","yearApproval":"year"}), on="year", how="left")    
)
#results["%automaticGE"] = results["automaticGE"]/results["automaticDrugId"]

results["%Direct GE between original target and original disease"] = results["Direct GE between original target and original disease"]/results["Approvals"]
results["%Indirect GE between original target and original disease"] = results["Indirect GE between original target and original disease"]/results["Approvals"]

# results["%Indirect GE between original/interacting target and original/related disease"] = results["Indirect GE between original/interacting target and original/related disease"]/results["Approvals"]
# results["%Indirect GE between original target and original/related disease"] = results["Indirect GE between original target and original/related disease"]/results["Approvals"]
# results["%Indirect GE between original/interacting target and original disease"] = results["Indirect GE between original/interacting target and original disease"]/results["Approvals"]

# results["%Direct GE between original/interacting target and original/related disease"] = results["Direct GE between original/interacting target and original/related disease"]/results["Approvals"]
# results["%Direct GE between original target and original/related disease"] = results["Direct GE between original target and original/related disease"]/results["Approvals"]
# results["%Direct GE between original/interacting target and original disease"] = results["Direct GE between original/interacting target and original disease"]/results["Approvals"]


ax = plt.figure().add_subplot()

# sns.lineplot(data=results, x="year", y="Indirect GE between original/interacting target and original disease", marker=".", markersize=10, color="orange", ax=ax, 
#              label="Indirect GE between original/interacting target and original disease")
# sns.lineplot(data=results, x="year", y="Indirect GE between original target and original/related disease", marker=".", markersize=10, color="purple", ax=ax, 
#              label="%Indirect GE between original/interacting target and original disease")
# sns.lineplot(data=results, x="year", y="%Indirect GE between original target and original disease", marker=".", markersize=10, color="b", ax=ax, 
#              label="%Indirect GE between original target and original disease")

# sns.lineplot(data=results, x="year", y="Direct GE between original/interacting target and original/related disease", marker=".", markersize=10, color="g", 
#              label="Direct GE between original/interacting target and original/related disease", ax=ax, linestyle="--")
# sns.lineplot(data=results, x="year", y="Direct GE between original/interacting target and original disease", marker=".", markersize=10, color="orange", ax=ax, linestyle="--", 
#              label="Direct GE between original/interacting target and original disease")
# sns.lineplot(data=results, x="year", y="Direct GE between original target and original/related disease", marker=".", markersize=10, color="purple", ax=ax, linestyle="--", 
#              label="Direct GE between original/interacting target and original disease")

sns.lineplot(data=results, x="year", y="%Direct GE between original target and original disease", marker=".", markersize=10, color=blue, ax=ax, linestyle="--", 
             label="%Direct GE between original target and original disease")
sns.lineplot(data=results, x="year", y="%Indirect GE between original target and original disease", marker=".", markersize=10, color=blue, ax=ax, linestyle="-", 
             label="%Indirect GE between original target and original disease")

ax.set_ylim(0,1)
ax.set_xlim(2013,2023)
ax.set_ylabel("% Approvals")
ax.set_xlabel("Approval year")
leg = ax.legend(loc='center left', bbox_to_anchor=(1, 0.5), frameon=0)
#plt.savefig("../results/23.02/plots/approvals.png", dpi=300, bbox_inches="tight")

Legends

In [ ]:
# timestamps legend
legend_elements = []
for source, color in {'Genetic association': '#577590',
 'Clinical': '#f94144',
 'Literature': '#90be6d',
 'Animal model': '#f9c74f',
 'Somatic mutations': '#43aa8b',
 'RNA expression': '#f8961e',
 'Affected pathway': '#f3722c',
 }.items():
    legend_elements.append(Patch(facecolor=color, edgecolor='k', label=source))

plt.figure()
fig, ax = plt.subplots()
ax.legend(
    handles=legend_elements,
    frameon=False,
    fontsize=plot_timeseries.font["size"],
    loc="center left",
    bbox_to_anchor=(1, 0.5),
    ncols=1,
    title="Category:"
)
plt.savefig("../results/23.06/plots/legends.png", dpi=300, bbox_inches="tight")

In [ ]:
# bells legend
plt.rcParams['legend.handlelength'] = 1.5
plt.rcParams['legend.handleheight'] = 1.51
legend_elements = []
for source in ["Human genetic data","RNA expression", "Animal model","Literature","Affected pathway",
               "Clinical phase I/II","Clinical phase III","Clinical phase IV"]:
    legend_elements.append(
    Line2D(
        [0],
        [0],
        color=plot_timeseries.getDatatypeToColour(by="label")[source],
        lw=5,
        label=source,
    )    
    )
plt.figure()
fig, ax = plt.subplots()
ax.legend(
    handles=legend_elements,
    frameon=False,
    fontsize=plot_timeseries.font["size"],
    loc="center left",
    bbox_to_anchor=(1, 0.5),
    ncols=4,
)
plt.savefig("../results/23.06/plots/legends.png", dpi=300, bbox_inches="tight")

In [ ]:
# therapy areas
plt.rcParams['legend.handlelength'] = .7
plt.rcParams['legend.handleheight'] = 1.51
#plt.rcParams['legend.numpoints'] = 1

tas = {idx: name for idx, name in plot_timeseries.getTherapeuticAreaToName().items() if name not in ('phenotype', 'injury, poisoning or other complication',"disorder of ear","hematologic disease","pregnancy or perinatal disease","psychiatric disorder")}
cmap_therapeuticAreas = dict(zip([tas[ta] for ta in sorted(tas, key=lambda x: tas[x])],
                                 list(sns.color_palette("tab20", len(tas)).as_hex())))
legend_elements = []
for source in sorted(cmap_therapeuticAreas):
    legend_elements.append(Patch(facecolor=cmap_therapeuticAreas[source], edgecolor='w', label=source.capitalize()))
plt.figure()
fig, ax = plt.subplots()
ax.legend(
    handles=legend_elements,
    frameon=False,
    fontsize=plot_timeseries.font["size"],
    loc="center left",
    bbox_to_anchor=(1, 0.5),
    ncols=2,
)
plt.savefig("../results/23.06/plots/legends.png", dpi=300, bbox_inches="tight")

In [ ]:
# correlations legend
plt.rcParams['legend.handlelength'] = .9
plt.rcParams['legend.handleheight'] = 1.51
legend_elements = []
for source in ["Affected pathway", "Animal model", "Clinical","Genetic association", "Literature","RNA expression","Somatic mutations"]:
    legend_elements.append(Patch(facecolor=plot_timeseries.getDatatypeToColour(by="label")[source], edgecolor='w', label=source))

plt.figure()
fig, ax = plt.subplots()
ax.legend(
    handles=legend_elements,
    frameon=False,
    fontsize=plot_timeseries.font["size"],
    loc="center left",
    bbox_to_anchor=(1, 0.5),
    ncols=1,
    title="Category:"
)
plt.savefig("../results/23.06/plots/legends.png", dpi=300, bbox_inches="tight")

Novels

In [ ]:
# novels heatmap
red = "#e40a36"
orange = "#ebb871"
green = "#44952a"

novels = pd.read_csv("/Users/mariaf/TargetEngine/results/23.06/novels/novels.csv").sort_values(["Therapeutic area","association"], ascending=[False, False]).set_index("association").T[::-1].T
repurposed = pd.read_csv("/Users/mariaf/TargetEngine/results/23.06/novels/repurposed.csv").sort_values(["Therapeutic area","association"], ascending=[False, False]).set_index("association").T[::-1].T
novels = pd.concat([repurposed,novels]).sort_values(["Therapeutic area","association"]).dropna(axis="columns", how="all")
#set1 = novels.head(50)#.dropna(axis="columns", how="all")
#set2 = novels.tail(50)#.dropna(axis="columns", how="all")

#for novels in (set1, set2):
if 1:
    drugs = novels[["Indication therapeutic area", "Drug"]]
    novels = (novels[["Therapeutic area"]+sorted([col for col in novels.columns if "novelty" in col])])
    novels = novels.rename(columns={col: col.replace(" novelty","") for col in novels.columns})

    data = novels.values.T

    cmap_datasources = LinearSegmentedColormap.from_list('cmap_datasources', ["white",'#2AAA69'], N=256)
    custom_colors = [(red, -1), (orange, 0), (green, 1)]
    cmap_target = ListedColormap([color for color, value in custom_colors], name='cmap_target')
    norm = plt.Normalize(min(value for color, value in custom_colors), max(value for color, value in custom_colors))
    tas = {idx: name for idx, name in plot_timeseries.getTherapeuticAreaToName().items() if name not in ('phenotype', 'injury, poisoning or other complication',"disorder of ear","hematologic disease","pregnancy or perinatal disease","psychiatric disorder")}
    cmap_therapeuticAreas = dict(zip([tas[ta] for ta in sorted(tas, key=lambda x: tas[x])],
                                    list(sns.color_palette("tab20", len(tas)).as_hex())))    
    cmap_therapeuticAreas[np.nan]="w"

    sns.set_style("white")
    fig, ax = plt.subplots(figsize=(30,25))

    cell_radius = 0.4
    square_size = 0.8  # Size of the square cells
    edge_color = 'gray'  # Color for the edge line

    for j in range(data.shape[1]):  
        for i in range(data.shape[0]):   
            if 0:
                pass                
            # if "Association score" in novels.columns[i]:
            #     square = Rectangle((i - square_size / 2, j - square_size / 2), square_size, square_size,
            #                        color=cmap_datasources(data[i, j]), edgecolor=edge_color, linewidth=0.5)
            #     ax.add_patch(square)
            #     if data[i, j] == 0:
            #         square.set_edgecolor(edge_color)
            #         square.set_linewidth(0.5) 
            
            if "Therapeutic area" in novels.columns[i]:  
                square = Rectangle((i , j - square_size / 2), .3, square_size,
                                color=cmap_therapeuticAreas[data[i, j]], edgecolor=edge_color, linewidth=0.5)
                ax.add_patch(square)
                if data[i, j] == 0:
                    square.set_edgecolor(edge_color)
                    square.set_linewidth(0.5)
            # elif "Drug" in novels.columns[i]:  
            #     ax.annotate(data[i,j], (i,j))
            else: # "novelty" in novels.columns[i]:
                circle = Circle((i, j), cell_radius, color=cmap_datasources(data[i, j]))
                ax.add_patch(circle)
                if (data[i, j] == 0) or (np.isnan(data[i, j])):
                    circle.set_edgecolor(edge_color)
                    circle.set_linewidth(0.2)                 

    ax.annotate("Drug for\nrepurposing", (i+3.2,j+1), fontsize=17, va="bottom", ha="center", annotation_clip=False)
    for idx, (ta, drug) in enumerate(drugs.values):
        if drug != drug:            
            txt = ""
        else:
            txt = drug.capitalize()
        if ta in ("congential disorder", "cancer or benign tumor","pancreas disease","endocrine system disease","immune system disease"):
            txt_colour = "w"
        else:
            txt_colour = "k"
        ax.annotate(txt, (i+3.2,idx), fontsize=16, color=txt_colour, va="center", ha="center", bbox=dict(boxstyle='round,pad=.2', facecolor=cmap_therapeuticAreas[ta]), annotation_clip=False)
                
    ax.set_aspect('equal')
    ax.set_frame_on(False)
    ax.set_xlim(-0.5, data.shape[0] - 0.5)  
    ax.set_ylim(-0.5, data.shape[1] - 0.5)
    ax.set_yticks(np.arange(data.shape[1]))  
    ax.set_yticklabels(novels.index)  
    ax.xaxis.tick_top()
    ax.set_xticks(np.arange(data.shape[0]))
    ax.set_xticklabels(novels.columns, rotation=50, ha="left")
    ax.xaxis.set_ticks_position('none')
    plt.savefig("../results/23.06/plots/novels.png", dpi=100, bbox_inches="tight")
    plt.show()


In [ ]:
# wordclouds
data = (spark.read.parquet("../results/23.06/novels/novels/novelty=0.4/year=2023")
        # .filter(~F.col("targetSymbol").isin(["CDCP1","TSPAN31","TMED3","NHIP","CASC20","DUSP5P1"]))
        .filter(~F.col("targetSymbol").isin(["TUFT1","CDCP1","BRCA1"]))
        #.filter(~F.col("therapeuticAreaName").isin(["reproductive system or breast disease","integumentary system disease"]))    
        .filter((F.col("datasources").contains("europepmc"))|(F.col("datasources").contains("ot_genetics_portal")))  
        .withColumn("metric", F.col("maxNovelty"))
        #.groupby("targetSymbol").agg(F.max("metric").alias("metric"))
        .orderBy("metric", ascending=False)
        #.limit(10)
        .withColumn("therapeuticAreaName", 
                    F.when(F.col("therapeuticAreaName")=="cancer or benign tumor", "Oncological diseases")
                    .otherwise(F.lit("Non-oncological diseases"))
        #             .when(F.col("therapeuticAreaName")=="psychiatric disorder", "Neuronal diseases")
        #             .when(F.col("therapeuticAreaName")=="musculoskeletal or connective tissue disease", "Musculoskeletal diseases")
        #             .otherwise("Other diseases")
         )
 ).toPandas()

target2score = dict(zip(data['targetSymbol'], data['metric']))

therapeuticAreaNames = sorted(data.datasources.unique())

def fixed_color_func(word, **kwargs):
    return "rgb(0, 0, 139)"  # Dark Blue


if 1:

    data_filtered=data
    
    # Create a dictionary of protein names and their corresponding scores
    target2score = dict(zip(data_filtered['targetSymbol'], data_filtered['metric']))
    
    # Generate the word cloud
    wordcloud = WordCloud(width=800, height=300, background_color='white', #color_func=blue_color_func,
                          font_path="/Library/Fonts/Roboto-Bold.ttf", 
                          colormap='Blues').generate_from_frequencies(target2score)
    
    # Plot the word cloud in the respective subplot
    plt.figure(figsize=(10, 5))
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.axis('off')  # No axis for word cloud display
    plt.savefig("../results/23.06/plots/wordclouds/all.png", dpi=500, bbox_inches="tight")

In [ ]:
# wordclouds
data = (spark.read.parquet("../results/23.06/novels/novels/novelty=0.4/year=2023")
        .filter(~F.col("targetSymbol").isin(["CDCP1","TSPAN31","TMED3","NHIP","CASC20","DUSP5P1"]))
        .filter(~F.col("targetSymbol").isin(["TUFT1","CDCP1","BRCA1"]))
        .filter(~F.col("therapeuticAreaName").isin(["reproductive system or breast disease","integumentary system disease"]))    
        
 ).toPandas()
data

In [ ]:
data=(spark.read.parquet("/Users/mariaf/TargetEngine/results/23.06/novels/novels/novelty=0.4/year=2023").select("targetIds").distinct().withColumnRenamed("targetIds","targetId")
.join(spark.read.parquet("/Users/mariaf/OT_platform/23.06/targetsPriorisation"), "targetId", "left")
.join(spark.read.parquet("/Users/mariaf/OT_platform/23.06/targets").select("biotype",F.col("id").alias("targetId")), "targetId", "left")

.fillna(0, subset=["maxClinicalTrialPhase","hasSafetyEvent"])
).toPandas()

In [ ]:
data

In [ ]:
data.groupby(["biotype"]).nunique().targetId

In [ ]:
data.groupby(["maxClinicalTrialPhase"]).nunique().targetId

In [ ]:
data.groupby(["hasSafetyEvent"]).nunique().targetId

In [ ]:
data.withColumn("a", F.max(["hasLigand","hasPocket"])).groupby(["a"]).nunique().targetId

In [ ]:
data[["hasLigand","hasPocket"]].max(axis=1).reset_index().rename(columns={0:"has"}).groupby(["has"]).nunique()#.index

In [ ]:
data.groupby(["hasPocket"]).nunique().targetId

In [ ]:
len(spark.read.parquet("/Users/mariaf/TargetEngine/results/23.06/novels/novels/novelty=0.4/year=2023").select("targetSymbol").distinct().toPandas())

In [ ]:
len(spark.read.parquet("/Users/mariaf/TargetEngine/results/23.06/novels/novels/novelty=0.4/year=2023").select("targetSymbol").distinct().toPandas())

In [ ]:
spark.read.parquet("/Users/mariaf/TargetEngine/results/23.06/novels/novels/novelty=0.4/year=2023").show()#.select("targetId").distinct())

In [ ]:
data

In [ ]:
spark.read.parquet("/Users/mariaf/OT_platform/23.06/targets").show()

In [ ]:
data

In [ ]:
data.groupby("maxClinicalTrialPhase").count()/1289*100

In [ ]:
data.groupby("hasSafetyEvent").count()#/1289*100

In [ ]:
data.groupby("hasLigand").count()#/1289*100

In [ ]:
data.groupby("hasPocket").count()#/1289*100

In [ ]:
data.groupby("hasHighQualityChemicalProbes").count()/1289*100

In [ ]:
# targets and associations found novel with different cutoffs
for cutoff in range(1,10)[::-1]:
    cutoff = cutoff/10
    associations = spark.read.parquet("/Users/mariaf/TargetEngine/results/23.06/novels/targets/novelty={}/year=2023".format(cutoff)).toPandas().sum().targets
    print(cutoff, associations)

In [ ]:
# bar chart: targetsByTherapeuticAreaName
data = (spark.read.parquet("../results/23.06/novels/targetsByTherapeuticAreaName0.4")
 .withColumnRenamed("novelty","targets")
 .replace({"cancer or benign tumor":"Oncological diseases",
           'nervous system disease':"Neuronal diseases",
           'musculoskeletal or connective tissue disease':"Musculoskeletal diseases",
           "genetic, familial or congenital disease":"Genetic diseases",
           "respiratory or thoracic disease":"Respiratory diseases",
           'integumentary system disease':"Integumentary diseases",
           'immune system disease':"Immune diseases",
           'reproductive system or breast disease':"Reproductive diseases",
           'nutritional or metabolic disease':"Metabolic diseases",
           'disorder of visual system':"Visual diseases",
           'urinary system disease':"Urinary diseases",
           "infectious disease":"Infectious diseases",
           "pancreas disease":"Pancreas diseases",
           "gastrointestinal disease":"Gastrointestinal diseases",
           "cardiovascular disease":"Cardiovascular diseases",
           })
 .orderBy("targets", ascending=False)
  .limit(15)
 ).toPandas()


plt.figure(figsize=(5,5)) 
ax = sns.barplot(data=data, x="therapeuticAreaName", y="targets", color=plot_timeseries.getDatatypeToColour()["genetic_association"])
ax.set_xticklabels(ax.get_xticklabels(), rotation=50, ha="right")
ax.set_xlabel("")
ax.set_ylabel("Unique targets\nwith novelty signal")
ax.set_title("Therapeutic area")
sns.despine()
plt.savefig("../results/23.06/plots/targetsByTherapeuticAreaName.png", dpi=300, bbox_inches="tight")

In [ ]:
# bar chart: targetsByDatasource
data = (spark.read.parquet("../results/23.06/novels/targetsByDatasource0.4")
 .withColumnRenamed("novelty","targets")
 .orderBy("targets", ascending=False)
  .limit(15)
 ).toPandas()

data["datatypeId"] = data.apply(lambda row: plot_timeseries.getDatatypeForDatasource(output="dict")[row.datasourceId], axis=1)
data = data.replace(plot_timeseries.getDatasourceToName(output="dict"))

fig = plt.figure(figsize=(5,5)) 
#fig.set_facecolor('w')
ax = sns.barplot(data=data, x="datasourceId", y="targets", hue="datatypeId", palette=plot_timeseries.getDatatypeToColour(), legend=False)
ax.set_xticklabels(ax.get_xticklabels(), rotation=50, ha="right")
ax.set_xlabel("")
ax.set_ylabel("Unique targets\nwith novelty signal")
ax.set_title("Novelty source")
sns.despine()
plt.savefig("../results/23.06/plots/targetsBySource.png", dpi=300, bbox_inches="tight")

In [ ]:
# bar chart: targetsByDatatype
data = (spark.read.parquet("../results/23.06/novels/targetsByDatatype0.4")
 .orderBy("targets", ascending=False)
  .limit(15)
 ).toPandas()

data = data.replace(plot_timeseries.getDatasourceToName(output="dict"))

fig = plt.figure(figsize=(5,5)) 
#fig.set_facecolor('w')
ax = sns.barplot(data=data, x="datatypeId", y="targets", hue="datatypeId", legend=False)
ax.set_xticklabels(ax.get_xticklabels(), rotation=50, ha="right")
ax.set_xlabel("")
ax.set_ylabel("Unique targets\nwith novelty signal")
ax.set_title("Novelty source")
sns.despine()
#plt.savefig("../results/23.06/plots/targetsBySource.png", dpi=300, bbox_inches="tight")

In [ ]:
data = (spark.read.parquet("../results/23.06/novels/associations/novelty=0.4/year=2023").orderBy("associations")
 ).toPandas()
data

In [ ]:
data = pd.read_csv("../results/23.06/novels/associationsWithNoveltyPeak2023.csv")
len(data.targetId.unique())

In [ ]:
spark.read.parquet("../results/23.06/novels/targetsByBiotype0.4").show()

In [ ]:
spark.read.parquet("/Users/mariaf/TargetEngine/results/23.06/novels/counts0.4/year=2021").toPandas().groupby(["datasourceId"]).sum().sort_values("associations")

In [ ]:
spark.read.parquet("/Users/mariaf/TargetEngine/results/23.06/novels/associationsByArea0.4/year=2023").toPandas().sum()

In [ ]:
spark.read.parquet("/Users/mariaf/TargetEngine/results/23.06/novels/associationsByArea/novelty=0.4/year=2023").show()#.toPandas().sum()

In [ ]:
pd.DataFrame([{"a":"a","b":"b"}])

In [ ]:
pd.concat([pd.DataFrame([{"a":"a","b":"b"}]),
           pd.DataFrame([{"a":"a","b":"b"}])])

In [ ]:
len(pd.read_csv("/Users/mariaf/TargetEngine/results/23.06/novels/associationsWithNoveltyPeak2023.csv")[["targetId","diseaseId"]].drop_duplicates())

In [ ]:
spark.read.parquet("/Users/mariaf/TargetEngine/results/23.06/novels/associationsByArea/novelty=0.4/year=2023").show()#.toPandas().sum()

In [ ]:
data = (spark.read.parquet("/Users/mariaf/TargetEngine/results/23.06/novels/counts0.4/year=2023")
        .drop("targets")
        #.filter(F.col("associations")>10)
 .replace({k:v+" diseases" for k, v in {"cancer or benign tumor":"Oncological",
           'nervous system disease':"Neuronal",
           'musculoskeletal or connective tissue disease':"Musculoskeletal",
           "genetic, familial or congenital disease":"Genetic",
           "respiratory or thoracic disease":"Respiratory",
           'integumentary system disease':"Integumentary",
           'immune system disease':"Immune",
           'reproductive system or breast disease':"Reproductive",
           'nutritional or metabolic disease':"Metabolic",
           'disorder of visual system':"Visual",
           'urinary system disease':"Urinary",
           "infectious disease":"Infectious",
           "pancreas disease":"Pancreas",
           "gastrointestinal disease":"Gastrointestinal",
           "cardiovascular disease":"Cardiovascular",
           "hematologic disease":"Hematologic",
           "pregnancy or perinatal disease":"Pegnancy",
           "psychiatric disorder":"Psyciatric",
           "endocrine system disease":"Endocrine",
           "disorder of ear":"Ear"
           }.items()})
 .orderBy("therapeuticAreaName","datasourceId")
 .drop("targets")
 ).toPandas().replace(plot_timeseries.getDatasourceToName(output="dict")).pivot(index="therapeuticAreaName", columns="datasourceId", values="associations")

In [ ]:
data.T.sum()

In [ ]:
plt.figure(figsize=(7,7))
ax = sns.heatmap(data.T, cmap="Blues", lw=1, vmax=100, linecolor="k", square=True, cbar=False)
ax.set_xticklabels(ax.get_xticklabels(), rotation=50, ha="right")
ax.set_title("Associations with novelty peaks in 2023")
ax.set_ylabel("Novelty source")
ax.set_xlabel("Therapeutic area")
plt.savefig("../results/23.06/plots/novels2023.png", bbox_inches="tight", dpi=300)

In [ ]:
data = (spark.read.parquet("/Users/mariaf/TargetEngine/results/23.06/novels/associationsByArea0.4")
        .filter(F.col("year")>=2005)
        .filter(F.col("year")<2023)
        .orderBy("year").replace("therapeuticAreaName","Oncological").replace("other","Non-oncological").toPandas()
        .pivot(index="year", columns="therapeuticAreaName", values="associations")
        
)

fig = plt.figure(figsize=(5,6))
ax = fig.add_subplot()
data.plot(kind='bar', stacked=True, width=.8, color=["#9CC5E1","#4280BE"], lw=1, edgecolor="k", ax=ax)
data = spark.read.parquet("/Users/mariaf/TargetEngine/results/23.06/novels/targets0.4").filter(F.col("year")>=2005).filter(F.col("year")<2023).orderBy("year").toPandas()
data[["targets"]].plot(kind="line", color="w", lw=6, ax=ax)
data[["targets"]].plot(kind="line", color="#FF595E", lw=4, ax=ax)
ax.set_xticklabels(ax.get_xticklabels(), rotation=90)
ax.set_title("Novelty peaks")
ax.set_xlabel("")
ax.set_ylim(0,30000)
ax.set_ylabel("Count")

ax.legend(
    handles=[Patch(facecolor="#4280BE", edgecolor='k', label="Oncological"), Patch(facecolor="#9CC5E1", edgecolor='k', label="Non-oncological"), Line2D([0], [0], color="#FF595E", lw=4, label="Unique targets")],
    frameon=False,
    title_fontsize=15,
    fontsize=15,
    loc="upper left",
    #bbox_to_anchor=(1, 0.5),
    ncols=1,
    title="Associations:"
)
plt.savefig("../results/23.06/plots/novels.png", dpi=300, bbox_inches="tight")

Extras

In [ ]:
# year difference for evidence with several timestamps
spark.read.parquet("../results/23.06/evidenceDated").select("publicationYear","curationYear","datasourceId").filter(F.col("publicationYear").isNotNull()&F.col("curationYear").isNotNull()).withColumn("window",F.col("curationYear")-F.col("publicationYear")).toPandas().groupby("datasourceId").median()

In [ ]:
["targetId","drugId","studyStartDate","urls"]

In [ ]:
spark.read.parquet("/Users/mariaf/OT_platform/24.09/searchDrug").show()

In [ ]:
pd.DataFrame({"1":[1,2,2],"2":[1,2,3]})

In [ ]:
(spark.read.parquet("/Users/mariaf/OT_platform/24.09/evidence/sourceId=chembl")
 .select(["targetId","drugId",F.explode("urls.niceName").alias("agency")])
 .filter(F.col("agency")=="FDA")
 .distinct()
).toPandas().to_csv("/Users/mariaf/DADs/data/approvalsOT.tsv",sep="\t",index=False)

In [ ]:
(
        spark.read.parquet("/Users/mariaf/OT_platform/24.09/targets")
        .withColumn("targetId",F.explode("proteinIds"))
        .filter(F.col("targetId.source") == "uniprot_swissprot")
        .select(
            F.col("targetId.id").alias("targetId"),
            F.col("approvedSymbol").alias("targetSymbol"),
            F.col("approvedName").alias("targetName"),
        )
        
        .distinct()
).printSchema()

In [ ]:
(spark.read.parquet("/Users/mariaf/OT_platform/24.09/evidence/sourceId=chembl")
 .select("targetId","diseaseId","datasourceId", "drugId","clinicalPhase","score","urls")
).show()

In [ ]:
pd.DataFrame([{"a":"a"}]).a.unique()

In [ ]:
spark.read.parquet("/Users/mariaf/OT_platform/24.09/interaction").select("sourceDatabase").distinct().show()

In [ ]:
spark.read.parquet("/Users/mariaf/OT_platform/24.09/interaction").show()

In [ ]:
spark.read.parquet("/Users/mariaf/OT_platform/25.03/credible_set").show()

Extra

In [ ]:
(spark.read.parquet("/Users/mariaf/OT_platform/25.03/sourceId=gwas_credible_sets").select("studyLocusId")
 .join(spark.read.parquet("/Users/mariaf/OT_platform/25.03/credible_set"), on="studyLocusId", how="left").select("studyId","studyLocusId")
 .join(spark.read.parquet("/Users/mariaf/OT_platform/25.03/study"), on="studyId", how="left")
 .select("studyId","studyLocusId","pubmedId","publicationDate")
).show()

In [ ]:
spark.read.parquet("/Users/mariaf/OT_platform/25.03/evidence").filter(F.col("publicationYear").isNull()&F.col("literature").isNotNull()).select("literature").distinct().count()#=="cancer_biomarkers").select("sourceId","literature","publicationYear").distinct().show()

In [ ]:
spark.read.parquet("/Users/mariaf/OT_platform/25.03/evidence").filter(F.col("datasourceId")=="cancer_biomarkers").select("sourceId","literature","publicationYear").distinct().show()

In [ ]:
spark.read.parquet("/Users/mariaf/TargetEngine/results/23.06/associationByDatasourceDirectOverYears/sourceId=expression_atlas").show()

In [ ]:
spark.read.parquet("/Users/mariaf/TargetEngine/results/23.06/evidenceDated/sourceId=europepmc").select("literature").printSchema()

In [ ]:
spark.read.parquet("/Users/mariaf/TargetEngine/results/23.06/evidenceDated/sourceId=europepmc").select("studyStartDate")

In [ ]:
data=(spark.read.parquet(
            "/Users/mariaf/OT_platform/25.03/evidence/sourceId=expression_atlas"
        )
        .select("studyId")
        .distinct()
        #.toPandas()
        #.studyId.tolist())
)

In [ ]:
data.count()

In [ ]:
spark.read.parquet("/Users/mariaf/OT_platform/25.03/evidence/sourceId=expression_atlas").filter(F.col("literature").isNull()).select("studyId","publicationYear","literature").show()

In [ ]:
spark.read.parquet("/Users/mariaf/OT_platform/23.06/evidence/sourceId=ot_genetics_portal").printSchema()

In [ ]:
spark.read.parquet("/Users/mariaf/OT_platform/25.03/sourceId=europepmc").withColumn("a", F.lit([123])).select("a").printSchema()#filter(F.col("literature").isNotNull()).select("literature").distinct().show()

In [ ]:
(spark.read.parquet("/Users/mariaf/OT_platform/25.03/evidence")
 .groupby("datasourceId").count().withColumnRenamed("count","evidence")
 .join(spark.read.parquet("/Users/mariaf/OT_platform/25.03/evidence").filter(F.col("studyStartDate").isNotNull()|F.col("publicationYear").isNotNull()).groupby("datasourceId").count().withColumnRenamed("count","timestamped_evidence"),
       on="datasourceId", how="left").fillna(0)
.join(spark.read.parquet("/Users/mariaf/OT_platform/25.03/evidence").filter(F.col("literature").isNotNull()).groupby("datasourceId").count().withColumnRenamed("count","literature_evidence"),
      on="datasourceId", how="left").fillna(0)
 #.select("datasourceId", F.col("id").alias("evidence"), F.least("publicationYear",F.split("studyStartDate","-").getItem(0).astype("int"))).show()
).toPandas()

In [ ]:
(spark.read.parquet("/Users/mariaf/OT_platform/25.03/evidence")
 .groupby("datasourceId").count().withColumnRenamed("count","evidence")
 .join(spark.read.parquet("/Users/mariaf/OT_platform/25.03/evidence").filter(F.col("studyStartDate").isNotNull()|F.col("publicationYear").isNotNull()).groupby("datasourceId").count().withColumnRenamed("count","timestamped_evidence"),
       on="datasourceId", how="left").fillna(0)
 #.select("datasourceId", F.col("id").alias("evidence"), F.least("publicationYear",F.split("studyStartDate","-").getItem(0).astype("int"))).show()
).toPandas()